In [20]:
import json
import numpy as np

## analysis

In [86]:
wnum=0
anum=0
with open("llama2_data/pubhealth+0.9/references_0.9_test.jsonl") as file:
    for line in file:
        anum+=1
        if "Claim: " not in line:
            wnum+=1
        #else:
        #    print(claim[1].encode().decode('unicode_escape'))
print(anum,wnum)

2018 23


In [76]:
with open("pubhealth/pubhealth_dev.json") as file1:
    for line1 in file1:
        data = json.loads(line1)
        claim1 = data['claim']
        anum+=1
        print(claim1)

Bat from Shawnee County tests positive for rabies.
Germany has banned pork from school canteens because it offends Muslim 'migrants.'
Coronavirus prompts Canada to roll out safe drugs for street users.
Wayne National Forest plans fires for tree, wildlife health.
School closures will have little impact on COVID-19 control, review finds.
The state of Michigan threatened local beavers with a $10,000 per day fine for failing to remove their dams.
More infected in fresh wave of China bird flu - WHO.
Raw Milk Straight from the Cow
“Maine House Democrats Vote to Allow Female Genital Mutilation.”
Trump: Asian-Americans not responsible for virus, need protection.
Kansas officials suggest alternatives to deal with pond scum.
Wholly domestic communication between you and your wife can go to New York to London and back and get caught up in the (NSA) database.
New Findings Show Xience Stents Are as Safe and Effective as Others
Italy looks to safeguard biomedical valley leading fight against COVID-1

In [83]:
mnum=0
nnum=0
xnum=0
anum=0
with open("pubhealth/pubhealth_dev.json") as file1:
    for line1 in file1:
        data = json.loads(line1)
        claim1 = data['claim']
        anum+=1
        with open("llama2_data/pubhealth+0.9/references_0.9_test.jsonl") as file:
            flag=0
            num=0
            for line in file:
                num+=1
                question=line.split("\"\"> Assistant: ")[0]
                claim=question.encode().decode('unicode_escape').split("Claim: ")
                if len(claim)==1:
                    claim=claim[0]
                    #print('wrong',num,claim)
                else:
                    claim=claim[1]
                if claim.strip()==claim1.strip():
                    flag+=1
            if flag==1:# flag==1:48;flag==2:961
                nnum+=1
                print(claim1)
            elif flag==2:
                mnum+=1
            else:
                xnum+=1
print(anum,nnum,mnum,xnum)

Bat from Shawnee County tests positive for rabies.
More infected in fresh wave of China bird flu - WHO.
Indonesian barber gears up to groom during coronavirus.
Canada has legalized euthanasia, enabling parents to "kill disabled children.
Selenium Supplements Might Give Modest Benefit Against Cholesterol
Raised for meat in South Korea, dogs head for new homes in U.S.
Every day, four North Carolinians die from a medication or drug overdose.
  Up to 70% of Americans have been infected with a “Vampire Fungus” from eating unhealthy foods that causes weight gain, skin problems and indigestion.   
J&J recalls 33,000 bottles of baby powder as FDA finds asbestos in sample.
Brain scans may help diagnose neurological, psychiatric disorders
Ohio’s stockpile got its start after the attacks of Sept. 11.
Coronavirus cases top half a million, protective gear lacking: WHO.
Hearing concludes in Missouri abortion clinic licensing case.
Evidence shows Zika virus turns fetus brains to liquid.
I lowered tax

## 实验 

In [21]:
numList=[202,207,192,245,239,242,175,177,184]#p9,p5,p1,w9,w5,w1,c1,c5,c9
num=[192]
filePath=['llama2_data/climate+0.9/','llama2_data/climate+0.5/','llama2_data/climate+0.1/',
          'llama2_data/pubhealth+0.9/','llama2_data/pubhealth+0.5/','llama2_data/pubhealth+0.1/',
          'llama2_data/wice+0.9/','llama2_data/wice+0.5/','llama2_data/wice+0.1/']

In [3]:
def identifyConflict(answer):
    if "here are no conflicting parts" in answer or"Conflicting parts not found" in answer or "here is no conflicting information" in answer or "no conflicting parts" in answer or "here is no conflict" in answer or "0 conflicts"in answer or "0 conflicting parts" in answer  or "0 Conflicting parts" in answer or "0 Conflicting parts found" in answer or "No conflict found" in answer or "P1 and P2 are not conflicting." in answer or"do not contain conflicting" in answer or "No\"" in answer or "0\"" in answer: 
        return True
    else:
        return False

In [83]:
def check(ref,ref1):
    if ref.strip()==ref1.strip():
        return True
    # 
    c1=ref.strip().split(".")
    c2=ref1.strip().split(".")
    if len(c1)>2 and len(c2)>2 and c1[-2]==c2[-2]:
        return True
    
    c1=ref.strip().split("\\\\n")[-1]
    c2=ref1.strip().split("\\\\n")[-1]
    if c1==c2:
        return True
    c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
    c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
    if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
        return True
    uni=ref.strip().split('\\u')[-1][4:]
    length=len(uni)
    if ref1.strip()[-length:] == uni:
        return True
    return False

# pubhealth+0.5

In [627]:
tlist=[[] for _ in range(13)]
flist=[[] for _ in range(13)]

In [628]:
for i in range(7):
    t=0
    f=0
    n=0
    x=0
    with open(filePath[4]+"dataset_0.5.jsonl") as file1:
        for line1 in file1:
            label=line1.split("label: ")[-1]
            ref1=line1.split("Response: ")[0].split("Reference: ")
            if len(ref1)==2:
                ref1=ref1[1]
            else:
                ref1=ref1[1]+"Reference: "+ref1[2]
            pre=n
            flag=False
            ref1=ref1[:-8]
            mark=[]
            if "Reference: " in ref1:
                ref1=ref1.split("Reference: ")[0]
            with open(filePath[4]+"detections_"+str(i+1)+".jsonl") as file:
                for line in file:
                    answer=line.split("Result: ")[1]
                    ref=line.split("P2: ")[0].split("P1: ")[1]
                    ref=ref.encode().decode('unicode_escape')
                    ref=ref[0:-8]
                    if ref.strip()==ref1.strip():
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                tlist[i].append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                flist[i].append(ref1)
       
                    if flag==False:
                        # 
                        c1=ref.strip().split(",")[0]
                        c2=ref1.strip().split(",")[0]
                        if c1.strip()==c2.strip() and "New Approach to Checking the Heart" not in ref and "Cuba sends doctors to Italy to help battle coronavirus" not in ref:
                            mark.append(',')
                            mark.append(ref)
                            flag=True
                            n+=1
                            if 'true' in label: #hallucination
                                
                                x+=1
                                if identifyConflict(answer): 
                                    t+=1
                                else:    
                                    tlist[i].append(ref1)
                            else: # factual
                                if identifyConflict(answer): 
                                    f+=1
                                else:    
                                    flist[i].append(ref1)
                    if flag==False:
                        # 
                        c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                        c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                        if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                            mark.append('"')
                            flag=True
                            n+=1
                            if 'true' in label: #hallucination
                                if identifyConflict(answer):
                                    t+=1
                                else:    
                                    tlist[i].append(ref1)
                            else: # factual
                                x+=1
                                if identifyConflict(answer):
                                    f+=1
                                else:    
                                    flist[i].append(ref1)
                    if flag==False:
                        if "\\\"Trump'"in ref and "\\\"Trump'" in ref1 and "after his presidential bid" not in ref and"after his presidential bi" not in ref1:
                            flag=True
                            n+=1
                            if 'true' in label: #hallucination
                                if identifyConflict(answer):
                                    t+=1
                                else:    
                                    tlist[i].append(ref1)
                            else: # factual
                                x+=1
                                if identifyConflict(answer):
                                    f+=1
                                else:    
                                    flist[i].append(ref1)
            if n-pre!=1:
                print(mark)
                print(n-pre)
                print(ref1)
                print()
    print(x)
    print(t)
    print(num[-1]-t,(num[-1]-t)/num[-1])
    print(f,f/num[-1])
    print('matched num=',n)

206
173
34 0.1642512077294686
185 0.893719806763285
matched num= 414
206
142
65 0.3140096618357488
165 0.7971014492753623
matched num= 414
206
169
38 0.18357487922705315
163 0.7874396135265701
matched num= 414
206
152
55 0.26570048309178745
183 0.8840579710144928
matched num= 414
206
153
54 0.2608695652173913
165 0.7971014492753623
matched num= 414
206
171
36 0.17391304347826086
185 0.893719806763285
matched num= 414
206
156
51 0.2463768115942029
180 0.8695652173913043
matched num= 414


In [629]:
for i in range(6):
    t=0
    f=0
    n=0
    x=0
    with open(filePath[4]+"dataset_0.5.jsonl") as file1:
        for line1 in file1:
            label=line1.split("label: ")[-1]
            ref1=line1.split("Response: ")[0].split("Reference: ")
            if len(ref1)==2:
                ref1=ref1[1]
            else:
                ref1=ref1[1]+"Reference: "+ref1[2]
            pre=n
            flag=False
            ref1=ref1[:-8]
            mark=[]
            if "Reference: " in ref1:
                ref1=ref1.split("Reference: ")[0]
            with open(filePath[4]+"detections_"+str(i+1)+"_1.jsonl") as file:
                for line in file:
                    answer=line.split("Result: ")[1]
                    ref=line.split("P2: ")[0].split("P1: ")[1]
                    ref=ref.encode().decode('unicode_escape')
                    ref=ref[0:-8]
                    if ref.strip()==ref1.strip():
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                tlist[i+7].append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                flist[i+7].append(ref1)
       
                    if flag==False:
                        # 
                        c1=ref.strip().split(",")[0]
                        c2=ref1.strip().split(",")[0]
                        if c1.strip()==c2.strip() and "New Approach to Checking the Heart" not in ref and "Cuba sends doctors to Italy to help battle coronavirus" not in ref:
                            mark.append(',')
                            mark.append(ref)
                            flag=True
                            n+=1
                            if 'true' in label: #hallucination
                                
                                x+=1
                                if identifyConflict(answer): 
                                    t+=1
                                else:    
                                    tlist[7+i].append(ref1)
                            else: # factual
                                if identifyConflict(answer): 
                                    f+=1
                                else:    
                                    flist[i+7].append(ref1)
                    if flag==False:
                        # 
                        c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                        c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                        if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                            mark.append('"')
                            flag=True
                            n+=1
                            if 'true' in label: #hallucination
                                if identifyConflict(answer):
                                    t+=1
                                else:    
                                    tlist[i+7].append(ref1)
                            else: # factual
                                x+=1
                                if identifyConflict(answer):
                                    f+=1
                                else:    
                                    flist[i+7].append(ref1)
                    if flag==False:
                        if "\\\"Trump'"in ref and "\\\"Trump'" in ref1 and "after his presidential bid" not in ref and"after his presidential bi" not in ref1:
                            flag=True
                            n+=1
                            if 'true' in label: #hallucination
                                if identifyConflict(answer):
                                    t+=1
                                else:    
                                    tlist[i+7].append(ref1)
                            else: # factual
                                x+=1
                                if identifyConflict(answer):
                                    f+=1
                                else:    
                                    flist[i+7].append(ref1)
            if n-pre!=1:
                print(mark)
                print(n-pre)
                print(ref1)
                print()
    print(x)
    print(t)
    print(num[-1]-t,(num[-1]-t)/num[-1])
    print(f,f/num[-1])
    print('matched num=',n)

206
176
31 0.1497584541062802
180 0.8695652173913043
matched num= 414
206
158
49 0.23671497584541062
173 0.8357487922705314
matched num= 414
206
173
34 0.1642512077294686
181 0.8743961352657005
matched num= 414
206
162
45 0.21739130434782608
185 0.893719806763285
matched num= 414
206
149
58 0.28019323671497587
157 0.7584541062801933
matched num= 414
206
149
58 0.28019323671497587
159 0.7681159420289855
matched num= 414


In [593]:
minV=0
for kk in range(13-0):
    #for jj in range(kk+1,13-0):
        #for ii in range(jj+1,13-0):
            #for i in range(ii+1,13-0):
                #for j in range(i+1,13-0):
                     #for k in range(j+1,13-0):
                        #for m in range(k+1,13-0):
                            #for n in range(m+1,13-0):
                                #for q in range(n+1,13-0):
                                    #for w in range(q+1,13-0):
                                        #for z in range(w+1,13-0):
                                            #for y in range(z+1,13-0):
                                                a=tlist[kk]#+tlist[jj]+tlist[ii]+tlist[i]+tlist[j]+tlist[k]+tlist[m]+tlist[n]+tlist[q]+tlist[w]+tlist[z]+tlist[y]+tlist[p] 
                                                a=list(set(a))
                                                aa=len(a)
                                                a=flist[kk]#+flist[jj]+flist[ii]+flist[i]+flist[j]+flist[k]+flist[m]+flist[n]+flist[q]+flist[w]+flist[z]+flist[y]+flist[p] 
                                                a=list(set(a))
                                                bb=len(a)
                                                TN=(num[-1]-bb)/(num[-1]*2)
                                                TP=aa/(num[-1]*2)
                                                FN=(num[-1]-aa)/(num[-1]*2)
                                                FP=bb/(num[-1]*2)
                                                F1=2*TP/(2*TP+FP+FN)
                                                if TN+TP>=minV:
                                                    print(kk,jj,ii,i)#,j,k,m,n,q,w,z,y,p)
                                                    print(TN+TP)
                                                    print(F1)
                                                    minV=TN+TP

0 12 12 12
0.5289855072463768
0.2585551330798479
1 12 12 12
0.5555555555555556
0.41401273885350315
3 12 12 12
0.5748792270531401
0.38461538461538464


In [630]:
a=t1+t2+t3+t4+t5+t6+t7+t11+t22+t33+t44+t55+t66 # 
res=set(a)
a=list(res)
print(len(a))
aa=len(a)
a=f1+f2+f3+f4+f5+f6+f7+f11+f22+f33+f44+f55+f66 # 
res=set(a)
a=list(res)
print(len(a))
bb=len(a)
TN=(num[-1]-bb)/(num[-1]*2)
TP=aa/(num[-1]*2)
FN=(num[-1]-aa)/(num[-1]*2)
FP=bb/(num[-1]*2)
F1=2*TP/(2*TP+FP+FN)
print(TN+TP)
print(F1)

149
150
0.4975845410628019
0.5889328063241107


# pubhealth+0.9

In [569]:
tlist=[[] for _ in range(13)]
flist=[[] for _ in range(13)]

In [570]:
for i in range(7):
    t=0
    f=0
    n=0
    x=0
    with open(filePath[3]+"dataset_0.9.jsonl") as file1:
        for line1 in file1:
            label=line1.split("label: ")[-1]
            ref1=line1.split("Response: ")[0].split("Reference: ")
            if len(ref1)==2:
                ref1=ref1[1]
            else:
                ref1=ref1[1]+"Reference: "+ref1[2]
            pre=n
            flag=False
            ref1=ref1[:-8]
            mark=[]
            if "Reference: " in ref1:
                ref1=ref1.split("Reference: ")[0]
            with open(filePath[3]+"detections_"+str(i+1)+".jsonl") as file:
                for line in file:
                    answer=line.split("Result: ")[1]
                    ref=line.split("P2: ")[0].split("P1: ")[1]
                    ref=ref.encode().decode('unicode_escape')
                    ref=ref[0:-8]
                    if ref.strip()==ref1.strip():
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                tlist[i].append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                flist[i].append(ref1)
       
                    if flag==False:
                        # 
                        c1=ref.strip().split(",")[0]
                        c2=ref1.strip().split(",")[0]
                        if c1.strip()==c2.strip() and "New Approach to Checking the Heart" not in ref and "Cuba sends doctors to Italy to help battle coronavirus" not in ref:
                            mark.append(',')
                            mark.append(ref)
                            flag=True
                            n+=1
                            if 'true' in label: #hallucination
                                
                                x+=1
                                if identifyConflict(answer): 
                                    t+=1
                                else:    
                                    tlist[i].append(ref1)
                            else: # factual
                                if identifyConflict(answer): 
                                    f+=1
                                else:    
                                    flist[i].append(ref1)
                    if flag==False:
                        # 
                        c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                        c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                        if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                            mark.append('"')
                            flag=True
                            n+=1
                            if 'true' in label: #hallucination
                                if identifyConflict(answer):
                                    t+=1
                                else:    
                                    tlist[i].append(ref1)
                            else: # factual
                                x+=1
                                if identifyConflict(answer):
                                    f+=1
                                else:    
                                    flist[i].append(ref1)

            if n-pre!=1:
                print(mark)
                print(n-pre)
                print(ref1)
                print()
    print(x)
    print(t)
    print(num[-1]-t,(num[-1]-t)/num[-1])
    print(f,f/num[-1])
    print('matched num=',n)

202
104
80 0.43478260869565216
125 0.6793478260869565
matched num= 404
202
102
82 0.44565217391304346
97 0.5271739130434783
matched num= 404
202
88
96 0.5217391304347826
118 0.6413043478260869
matched num= 404
202
89
95 0.5163043478260869
97 0.5271739130434783
matched num= 404
202
103
81 0.44021739130434784
121 0.657608695652174
matched num= 404
202
123
61 0.33152173913043476
143 0.7771739130434783
matched num= 404
202
111
73 0.3967391304347826
123 0.6684782608695652
matched num= 404


In [571]:
for i in range(6):
    t=0
    f=0
    n=0
    x=0
    with open(filePath[3]+"dataset_0.9.jsonl") as file1:
        for line1 in file1:
            label=line1.split("label: ")[-1]
            ref1=line1.split("Response: ")[0].split("Reference: ")
            if len(ref1)==2:
                ref1=ref1[1]
            else:
                ref1=ref1[1]+"Reference: "+ref1[2]
            pre=n
            flag=False
            ref1=ref1[:-8]
            mark=[]
            if "Reference: " in ref1:
                ref1=ref1.split("Reference: ")[0]
            with open(filePath[3]+"detections_"+str(i+1)+"_1.jsonl") as file:
                for line in file:
                    answer=line.split("Result: ")[1]
                    ref=line.split("P2: ")[0].split("P1: ")[1]
                    ref=ref.encode().decode('unicode_escape')
                    ref=ref[0:-8]
                    if ref.strip()==ref1.strip():
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                tlist[i+7].append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                flist[i+7].append(ref1)
       
                    if flag==False:
                        # 
                        c1=ref.strip().split(",")[0]
                        c2=ref1.strip().split(",")[0]
                        if c1.strip()==c2.strip() and "New Approach to Checking the Heart" not in ref and "Cuba sends doctors to Italy to help battle coronavirus" not in ref:
                            mark.append(',')
                            mark.append(ref)
                            flag=True
                            n+=1
                            if 'true' in label: #hallucination
                                
                                x+=1
                                if identifyConflict(answer): 
                                    t+=1
                                else:    
                                    tlist[i+7].append(ref1)
                            else: # factual
                                if identifyConflict(answer): 
                                    f+=1
                                else:    
                                    flist[i+7].append(ref1)
                    if flag==False:
                        # 
                        c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                        c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                        if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                            mark.append('"')
                            flag=True
                            n+=1
                            if 'true' in label: #hallucination
                                if identifyConflict(answer):
                                    t+=1
                                else:    
                                    tlist[i+7].append(ref1)
                            else: # factual
                                x+=1
                                if identifyConflict(answer):
                                    f+=1
                                else:    
                                    flist[i+7].append(ref1)

            if n-pre!=1:
                print(mark)
                print(n-pre)
                print(ref1)
                print()
    print(x)
    print(t)
    print(num[-1]-t,(num[-1]-t)/num[-1])
    print(f,f/num[-1])
    print('matched num=',n)

202
120
64 0.34782608695652173
126 0.6847826086956522
matched num= 404
202
122
62 0.33695652173913043
128 0.6956521739130435
matched num= 404
202
128
56 0.30434782608695654
144 0.782608695652174
matched num= 404
202
131
53 0.28804347826086957
134 0.7282608695652174
matched num= 404
202
105
79 0.42934782608695654
118 0.6413043478260869
matched num= 404
202
113
71 0.3858695652173913
117 0.6358695652173914
matched num= 404


In [575]:
minV=0
for kk in range(13-11):
    for jj in range(kk+1,13-10):
        for ii in range(jj+1,13-9):
            for i in range(ii+1,13-8):
                for j in range(i+1,13-7):
                     for k in range(j+1,13-6):
                        for m in range(k+1,13-5):
                            for n in range(m+1,13-4):
                                for q in range(n+1,13-3):
                                    for w in range(q+1,13-2):
                                        for z in range(w+1,13-1):
                                            for y in range(z+1,13-0):
                                                a=tlist[kk]+tlist[jj]+tlist[ii]+tlist[i]+tlist[j]+tlist[k]+tlist[m]+tlist[n]+tlist[q]+tlist[w]+tlist[z]+tlist[y]#+tlist[p] 
                                                a=list(set(a))
                                                aa=len(a)
                                                a=flist[kk]+flist[jj]+flist[ii]+flist[i]+flist[j]+flist[k]+flist[m]+flist[n]+flist[q]+flist[w]+flist[z]+flist[y]#+flist[p] 
                                                a=list(set(a))
                                                bb=len(a)
                                                TN=(num[-1]-bb)/(num[-1]*2)
                                                TP=aa/(num[-1]*2)
                                                FN=(num[-1]-aa)/(num[-1]*2)
                                                FP=bb/(num[-1]*2)
                                                F1=2*TP/(2*TP+FP+FN)
                                                if TN+TP>=minV:
                                                    print(kk,jj,ii,i)#,j,k,m,n,q,w,z,y,p)
                                                    print(TN+TP)
                                                    print(F1)
                                                    minV=TN+TP

0 1 2 3
0.504950495049505
0.6677740863787376
0 1 2 3
0.504950495049505
0.6677740863787376
0 1 2 3
0.504950495049505
0.6677740863787376
0 1 2 3
0.504950495049505
0.6677740863787376
0 1 2 3
0.504950495049505
0.6666666666666666
0 2 3 4
0.504950495049505
0.6677740863787376
1 2 3 4
0.5074257425742574
0.667779632721202


In [594]:
a=t1+t2+t3+t4+t5+t6+t7+t11+t22+t33+t44+t55+t66 # 
res=set(a)
a=list(res)
print(len(a))
aa=len(a)
a=f1+f2+f3+f4+f5+f6+f7+f11+f22+f33+f44+f55+f66 # 
res=set(a)
a=list(res)
print(len(a))
bb=len(a)
TN=(num[-1]-bb)/(num[-1]*2)
TP=aa/(num[-1]*2)
FN=(num[-1]-aa)/(num[-1]*2)
FP=bb/(num[-1]*2)
F1=2*TP/(2*TP+FP+FN)
print(TN+TP)
print(F1)

176
179
0.49275362318840576
0.6263345195729538


# pubhealth+0.1

In [22]:
tlist=[[] for _ in range(13)]
flist=[[] for _ in range(13)]

In [23]:
for i in range(7):
    t=0
    f=0
    n=0
    x=0
    with open(filePath[5]+"dataset_0.1.jsonl") as file1:
        for line1 in file1:
            label=line1.split("label: ")[-1]
            ref1=line1.split("Response: ")[0].split("Reference: ")
            if len(ref1)==2:
                ref1=ref1[1]
            else:
                ref1=ref1[1]+"Reference: "+ref1[2]
            pre=n
            flag=False
            ref1=ref1[:-8]
            mark=[]
            if "Reference: " in ref1:
                ref1=ref1.split("Reference: ")[0]
            with open(filePath[5]+"detections_"+str(i+1)+".jsonl") as file:
                for line in file:
                    answer=line.split("Result: ")[1]
                    ref=line.split("P2: ")[0].split("P1: ")[1]
                    ref=ref.encode().decode('unicode_escape')
                    ref=ref[0:-8]
                    if ref.strip()==ref1.strip():
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                tlist[i].append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                flist[i].append(ref1)
                    if flag==False:
                        # 
                        c1=ref.strip().split(",")[0]
                        c2=ref1.strip().split(",")[0]
                        if c1.strip()==c2.strip():
                            flag=True
                            n+=1
                            if 'true' in label: #hallucination
                                x+=1
                                if identifyConflict(answer): 
                                    t+=1
                                else:    
                                    tlist[i].append(ref1)
                            else: # factual
                                if identifyConflict(answer): 
                                    f+=1
                                else:    
                                    flist[i].append(ref1)
                    if flag==False:
                        # 
                        c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                        c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                        if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                            flag=True
                            n+=1
                            if 'true' in label: #hallucination
                                if identifyConflict(answer):
                                    t+=1
                                else:    
                                    tlist[i].append(ref1)
                            else: # factual
                                x+=1
                                if identifyConflict(answer):
                                    f+=1
                                else:    
                                    flist[i].append(ref1)
                    if flag==False:
                        if ("The Economist magazine (2015)" in ref and "The Economist magazine (2015)" in ref1) or ("2018 Academy Awards Nominations." in ref and "2018 Academy Awards Nominations." in ref1) or ("538 RSS feed for " in ref and "538 RSS feed for " in ref1) or ("2018 Annual Report of Dignity Heal" in ref and "2018 Annual Report of Dignity Heal" in ref1) or ("DVD commentary (Creators/Writers Chris Carter, Vince Gilligan" in ref and "DVD commentary (Creators/Writers Chris Carter, Vince Gilligan" in ref1) or ("247sports.com Summary: According to a report by 247Sports" in ref and "247sports.com Summary: According to a report by 247Sports" in ref1) or ("The phpBB post by JtRx13 who made the first post on May 6, 2009" in ref and "The phpBB post by JtRx13 who made the first post on May 6, 2009" in ref1) or ("The Ospreys overcame a late rally by Newcastle to win 17" in ref and "The Ospreys overcame a late rally by Newcastle to win 17" in ref1):
                            flag=True
                            n+=1
                            if 'true' in label: #hallucination
                                if identifyConflict(answer):
                                    t+=1
                                else:    
                                    tlist[i].append(ref1)
                            else: # factual
                                x+=1
                                if identifyConflict(answer):
                                    f+=1
                                else:    
                                    flist[i].append(ref1)
            if n-pre!=1:
                print(mark)
                print(n-pre)
                print(ref1)
                print()
    print(x)
    print(t)
    print(num[-1]-t,(num[-1]-t)/num[-1])
    print(f,f/num[-1])
    print('matched num=',n)

192
179
13 0.06770833333333333
182 0.9479166666666666
matched num= 384
192
159
33 0.171875
163 0.8489583333333334
matched num= 384
192
172
20 0.10416666666666667
170 0.8854166666666666
matched num= 384
192
162
30 0.15625
176 0.9166666666666666
matched num= 384
192
163
29 0.15104166666666666
157 0.8177083333333334
matched num= 384
192
172
20 0.10416666666666667
181 0.9427083333333334
matched num= 384
192
170
22 0.11458333333333333
172 0.8958333333333334
matched num= 384


In [24]:
for i in range(6):
    t=0
    f=0
    n=0
    x=0
    with open(filePath[5]+"dataset_0.1.jsonl") as file1:
        for line1 in file1:
            label=line1.split("label: ")[-1]
            ref1=line1.split("Response: ")[0].split("Reference: ")
            if len(ref1)==2:
                ref1=ref1[1]
            else:
                ref1=ref1[1]+"Reference: "+ref1[2]
            pre=n
            flag=False
            ref1=ref1[:-8]
            mark=[]
            if "Reference: " in ref1:
                ref1=ref1.split("Reference: ")[0]
            with open(filePath[5]+"detections_"+str(i+1)+"_1.jsonl") as file:
                for line in file:
                    answer=line.split("Result: ")[1]
                    ref=line.split("P2: ")[0].split("P1: ")[1]
                    ref=ref.encode().decode('unicode_escape')
                    ref=ref[0:-8]
                    if ref.strip()==ref1.strip():
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                tlist[i+7].append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                flist[i+7].append(ref1)
                    if flag==False:
                        # 
                        c1=ref.strip().split(",")[0]
                        c2=ref1.strip().split(",")[0]
                        if c1.strip()==c2.strip():
                            flag=True
                            n+=1
                            if 'true' in label: #hallucination
                                x+=1
                                if identifyConflict(answer): 
                                    t+=1
                                else:    
                                    tlist[i+7].append(ref1)
                            else: # factual
                                if identifyConflict(answer): 
                                    f+=1
                                else:    
                                    flist[7+i].append(ref1)
                    if flag==False:
                        # 
                        c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                        c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                        if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                            flag=True
                            n+=1
                            if 'true' in label: #hallucination
                                if identifyConflict(answer):
                                    t+=1
                                else:    
                                    tlist[i+7].append(ref1)
                            else: # factual
                                x+=1
                                if identifyConflict(answer):
                                    f+=1
                                else:    
                                    flist[i+7].append(ref1)
                    if flag==False:
                        if ("The Economist magazine (2015)" in ref and "The Economist magazine (2015)" in ref1) or ("2018 Academy Awards Nominations." in ref and "2018 Academy Awards Nominations." in ref1) or ("538 RSS feed for " in ref and "538 RSS feed for " in ref1) or ("2018 Annual Report of Dignity Heal" in ref and "2018 Annual Report of Dignity Heal" in ref1) or ("DVD commentary (Creators/Writers Chris Carter, Vince Gilligan" in ref and "DVD commentary (Creators/Writers Chris Carter, Vince Gilligan" in ref1) or ("247sports.com Summary: According to a report by 247Sports" in ref and "247sports.com Summary: According to a report by 247Sports" in ref1) or ("The phpBB post by JtRx13 who made the first post on May 6, 2009" in ref and "The phpBB post by JtRx13 who made the first post on May 6, 2009" in ref1) or ("The Ospreys overcame a late rally by Newcastle to win 17" in ref and "The Ospreys overcame a late rally by Newcastle to win 17" in ref1):
                            flag=True
                            n+=1
                            if 'true' in label: #hallucination
                                if identifyConflict(answer):
                                    t+=1
                                else:    
                                    tlist[i+7].append(ref1)
                            else: # factual
                                x+=1
                                if identifyConflict(answer):
                                    f+=1
                                else:    
                                    flist[i+7].append(ref1)
            if n-pre!=1:
                print(mark)
                print(n-pre)
                print(ref1)
                print()
    print(x)
    print(t)
    print(num[-1]-t,(num[-1]-t)/num[-1])
    print(f,f/num[-1])
    print('matched num=',n)

192
171
21 0.109375
178 0.9270833333333334
matched num= 384
192
174
18 0.09375
173 0.9010416666666666
matched num= 384
192
179
13 0.06770833333333333
179 0.9322916666666666
matched num= 384
192
154
38 0.19791666666666666
169 0.8802083333333334
matched num= 384
192
138
54 0.28125
139 0.7239583333333334
matched num= 384
192
141
51 0.265625
152 0.7916666666666666
matched num= 384


In [26]:
minV=0
for kk in range(13-2):
    for jj in range(kk+1,13-1):
        for ii in range(jj+1,13-0):
            #for i in range(ii+1,13-1):
                #for j in range(i+1,13-0):
                     #for k in range(j+1,13-4):
                        #for m in range(k+1,13-3):
                            #for n in range(m+1,13-2):
                                #for q in range(n+1,13-1):
                                    #for w in range(q+1,13-0):
                                        #for z in range(w+1,13-0):
                                            #for y in range(z+1,13-0):
                                                a=tlist[kk]+tlist[jj]+tlist[ii]#+tlist[i]+tlist[j]#+tlist[k]+tlist[m]+tlist[n]+tlist[q]+tlist[w]#+tlist[z]+tlist[y]+tlist[p] 
                                                a=list(set(a))
                                                aa=len(a)
                                                a=flist[kk]+flist[jj]+flist[ii]#+flist[i]+flist[j]#+flist[k]+flist[m]+flist[n]+flist[q]+flist[w]#+flist[z]+flist[y]+flist[p] 
                                                a=list(set(a))
                                                bb=len(a)
                                                TN=(num[-1]-bb)/(num[-1]*2)
                                                TP=aa/(num[-1]*2)
                                                FN=(num[-1]-aa)/(num[-1]*2)
                                                FP=bb/(num[-1]*2)
                                                F1=2*TP/(2*TP+FP+FN)
                                                if TN+TP>=minV:
                                                    print(kk,jj,ii,i)#,j,k,m,n,q,w,z,y,p)
                                                    print(TN+TP)
                                                    print(F1)
                                                    minV=TN+TP

0 1 2 11
0.49739583333333337
0.35451505016722407
0 1 3 11
0.5520833333333334
0.43790849673202614
0 3 5 11
0.5546875
0.3643122676579926
0 3 10 11
0.5651041666666667
0.44147157190635455
1 3 10 11
0.5677083333333334
0.4876543209876543
3 5 10 11
0.5729166666666666
0.45333333333333337


In [19]:
a=[]
for i in range(13):
    a+=tlist[i]
res=set(a)
a=list(res)
print(len(a))
aa=len(a)
a=[]
for i in range(13):
    a+=flist[i]
res=set(a)
a=list(res)
print(len(a))
bb=len(a)
TN=(num[-1]-bb)/(num[-1]*2)
TP=aa/(num[-1]*2)
FN=(num[-1]-aa)/(num[-1]*2)
FP=bb/(num[-1]*2)
F1=2*TP/(2*TP+FP+FN)
print(TN+TP)
print(F1)

148
142
0.515625
0.6141078838174274


# wice+0.9

In [396]:
tlist=[[] for _ in range(13)]
flist=[[] for _ in range(13)]

In [397]:
for i in range(7):
    t=0
    f=0
    n=0
    x=0
    with open(filePath[6]+"dataset_0.9.jsonl") as file1:
        for line1 in file1:
            label=line1.split("label: ")[-1]
            ref1=line1.split("Response: ")[0].split("Reference: ")
            if len(ref1)==2:
                ref1=ref1[1]
            else:
                ref1=ref1[1]+"Reference: "+ref1[2]
            pre=n
            flag=False
            ref1=ref1[:-8]
            mark=[]
            if "Reference: " in ref1:
                ref1=ref1.split("Reference: ")[0]
            with open(filePath[6]+"detections_"+str(i+1)+".jsonl") as file:
                for line in file:
                    answer=line.split("Result: ")[1]
                    ref=line.split("P2: ")[0].split("P1: ")[1]
                    ref=ref.encode().decode('unicode_escape')
                    ref=ref[0:-8]
                    if ref.strip()==ref1.strip():
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                tlist[i].append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                flist[i].append(ref1)
                    if flag==False:
                        # 
                        c1=ref.strip().split(",")[0]
                        c2=ref1.strip().split(",")[0]
                        if c1.strip()==c2.strip():
                            flag=True
                            n+=1
                            if 'true' in label: #hallucination
                                x+=1
                                if identifyConflict(answer): 
                                    t+=1
                                else:    
                                    tlist[i].append(ref1)
                            else: # factual
                                if identifyConflict(answer): 
                                    f+=1
                                else:    
                                    flist[i].append(ref1)
                    if flag==False:
                        # 
                        c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                        c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                        if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                            flag=True
                            n+=1
                            if 'true' in label: #hallucination
                                if identifyConflict(answer):
                                    t+=1
                                else:    
                                    tlist[i].append(ref1)
                            else: # factual
                                x+=1
                                if identifyConflict(answer):
                                    f+=1
                                else:    
                                    flist[i].append(ref1)
                    if flag==False:
                        if ("The Economist magazine (2015)" in ref and "The Economist magazine (2015)" in ref1) or ("2018 Academy Awards Nominations." in ref and "2018 Academy Awards Nominations." in ref1) or ("538 RSS feed for " in ref and "538 RSS feed for " in ref1) or ("2018 Annual Report of Dignity Heal" in ref and "2018 Annual Report of Dignity Heal" in ref1) or ("DVD commentary (Creators/Writers Chris Carter, Vince Gilligan" in ref and "DVD commentary (Creators/Writers Chris Carter, Vince Gilligan" in ref1) or ("247sports.com Summary: According to a report by 247Sports" in ref and "247sports.com Summary: According to a report by 247Sports" in ref1) or ("The phpBB post by JtRx13 who made the first post on May 6, 2009" in ref and "The phpBB post by JtRx13 who made the first post on May 6, 2009" in ref1) or ("The Ospreys overcame a late rally by Newcastle to win 17" in ref and "The Ospreys overcame a late rally by Newcastle to win 17" in ref1):
                            flag=True
                            n+=1
                            if 'true' in label: #hallucination
                                if identifyConflict(answer):
                                    t+=1
                                else:    
                                    tlist[i].append(ref1)
                            else: # factual
                                x+=1
                                if identifyConflict(answer):
                                    f+=1
                                else:    
                                    flist[i].append(ref1)
            if n-pre!=1:
                print(mark)
                print(n-pre)
                print(ref1)
                print()
    print(x)
    print(t)
    print(num[-1]-t,(num[-1]-t)/num[-1])
    print(f,f/num[-1])
    print('matched num=',n)

244
186
59 0.24081632653061225
183 0.746938775510204
matched num= 490
244
150
95 0.3877551020408163
162 0.6612244897959184
matched num= 490
244
113
132 0.5387755102040817
120 0.4897959183673469
matched num= 490
244
126
119 0.4857142857142857
159 0.6489795918367347
matched num= 490
244
128
117 0.4775510204081633
146 0.5959183673469388
matched num= 490
244
144
101 0.4122448979591837
154 0.6285714285714286
matched num= 490
244
130
115 0.46938775510204084
136 0.5551020408163265
matched num= 490


In [398]:
for i in range(6):
    t=0
    f=0
    n=0
    x=0
    with open(filePath[6]+"dataset_0.9.jsonl") as file1:
        for line1 in file1:
            label=line1.split("label: ")[-1]
            ref1=line1.split("Response: ")[0].split("Reference: ")
            if len(ref1)==2:
                ref1=ref1[1]
            else:
                ref1=ref1[1]+"Reference: "+ref1[2]
            pre=n
            flag=False
            ref1=ref1[:-8]
            mark=[]
            if "Reference: " in ref1:
                ref1=ref1.split("Reference: ")[0]
            with open(filePath[6]+"detections_"+str(i+1)+"_1.jsonl") as file:
                for line in file:
                    answer=line.split("Result: ")[1]
                    ref=line.split("P2: ")[0].split("P1: ")[1]
                    ref=ref.encode().decode('unicode_escape')
                    ref=ref[0:-8]
                    if ref.strip()==ref1.strip():
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                tlist[i+7].append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                flist[i+7].append(ref1)
                    if flag==False:
                        # 
                        c1=ref.strip().split(",")[0]
                        c2=ref1.strip().split(",")[0]
                        if c1.strip()==c2.strip():
                            flag=True
                            n+=1
                            if 'true' in label: #hallucination
                                x+=1
                                if identifyConflict(answer): 
                                    t+=1
                                else:    
                                    tlist[i+7].append(ref1)
                            else: # factual
                                if identifyConflict(answer): 
                                    f+=1
                                else:    
                                    flist[i+7].append(ref1)
                    if flag==False:
                        # 
                        c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                        c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                        if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                            flag=True
                            n+=1
                            if 'true' in label: #hallucination
                                if identifyConflict(answer):
                                    t+=1
                                else:    
                                    tlist[i+7].append(ref1)
                            else: # factual
                                x+=1
                                if identifyConflict(answer):
                                    f+=1
                                else:    
                                    flist[i+7].append(ref1)
                    if flag==False:
                        if ("The Economist magazine (2015)" in ref and "The Economist magazine (2015)" in ref1) or ("2018 Academy Awards Nominations." in ref and "2018 Academy Awards Nominations." in ref1) or ("538 RSS feed for " in ref and "538 RSS feed for " in ref1) or ("2018 Annual Report of Dignity Heal" in ref and "2018 Annual Report of Dignity Heal" in ref1) or ("DVD commentary (Creators/Writers Chris Carter, Vince Gilligan" in ref and "DVD commentary (Creators/Writers Chris Carter, Vince Gilligan" in ref1) or ("247sports.com Summary: According to a report by 247Sports" in ref and "247sports.com Summary: According to a report by 247Sports" in ref1) or ("The phpBB post by JtRx13 who made the first post on May 6, 2009" in ref and "The phpBB post by JtRx13 who made the first post on May 6, 2009" in ref1) or ("The Ospreys overcame a late rally by Newcastle to win 17" in ref and "The Ospreys overcame a late rally by Newcastle to win 17" in ref1):
                            flag=True
                            n+=1
                            if 'true' in label: #hallucination
                                if identifyConflict(answer):
                                    t+=1
                                else:    
                                    tlist[i+7].append(ref1)
                            else: # factual
                                x+=1
                                if identifyConflict(answer):
                                    f+=1
                                else:    
                                    flist[i+7].append(ref1)
            if n-pre!=1:
                print(mark)
                print(n-pre)
                print(ref1)
                print()
    print(x)
    print(t)
    print(num[-1]-t,(num[-1]-t)/num[-1])
    print(f,f/num[-1])
    print('matched num=',n)

244
131
114 0.46530612244897956
140 0.5714285714285714
matched num= 490
244
171
74 0.3020408163265306
174 0.710204081632653
matched num= 490
244
204
41 0.1673469387755102
205 0.8367346938775511
matched num= 490
244
156
89 0.363265306122449
177 0.7224489795918367
matched num= 490
244
112
133 0.5428571428571428
101 0.4122448979591837
matched num= 490
244
143
102 0.4163265306122449
155 0.6326530612244898
matched num= 490


In [404]:
minV=0
for kk in range(13-7):
    for jj in range(kk+1,13-6):
        for ii in range(jj+1,13-5):
            for i in range(ii+1,13-4):
                for j in range(i+1,13-3):
                     for k in range(j+1,13-2):
                        for m in range(k+1,13-1):
                            for n in range(m+1,13-0):
                                #for q in range(n+1,13-0):
                                    #for w in range(q+1,13-0):
                                        #for z in range(w+1,13-0):
                                            #for y in range(z+1,13-0):
                                                a=tlist[kk]+tlist[jj]+tlist[ii]+tlist[i]+tlist[j]+tlist[k]+tlist[m]+tlist[n]#+tlist[q]+tlist[w]+tlist[z]+tlist[y]+tlist[p] 
                                                a=list(set(a))
                                                aa=len(a)
                                                a=flist[kk]+flist[jj]+flist[ii]+flist[i]+flist[j]+flist[k]+flist[m]+flist[n]#+flist[q]+flist[w]+flist[z]+flist[y]+flist[p] 
                                                a=list(set(a))
                                                bb=len(a)
                                                TN=(num[-1]-bb)/(num[-1]*2)
                                                TP=aa/(num[-1]*2)
                                                FN=(num[-1]-aa)/(num[-1]*2)
                                                FP=bb/(num[-1]*2)
                                                F1=2*TP/(2*TP+FP+FN)
                                                if TN+TP>=minV:
                                                    print(kk,jj,ii,i)#,j,k,m,n,q,w,z,y,p)
                                                    print(TN+TP)
                                                    print(F1)
                                                    minV=TN+TP

0 1 2 3
0.4938775510204082
0.6565096952908587
0 1 2 3
0.5
0.6611341632088521
0 1 2 3
0.5
0.6573426573426573
0 1 2 3
0.5020408163265306
0.6601671309192201
0 1 2 3
0.5020408163265306
0.6582633053221288
0 1 2 3
0.5040816326530613
0.6620305980528511
0 1 2 3
0.5081632653061224
0.6666666666666666
0 1 2 3
0.5122448979591837
0.6609929078014184
0 1 2 3
0.5122448979591837
0.6647966339410939
0 1 2 3
0.5122448979591837
0.6657342657342658
0 1 2 4
0.5142857142857142
0.6647887323943662
0 1 2 4
0.5142857142857143
0.6694444444444444
0 1 2 4
0.5163265306122449
0.6712898751733702
0 1 2 4
0.5163265306122449
0.667601683029453
0 1 3 5
0.5163265306122449
0.6657263751763046
0 1 3 5
0.5183673469387755
0.659942363112392
0 1 3 5
0.5183673469387755
0.6609195402298851
0 1 3 7
0.5183673469387755
0.659942363112392
0 1 4 5
0.5183673469387755
0.6657223796033994
0 1 4 6
0.5224489795918368
0.6685552407932012
0 4 5 6
0.5224489795918368
0.6666666666666666


# wice+0.5

In [284]:
tlist=[[] for _ in range(13)]
flist=[[] for _ in range(13)]

In [285]:
for i in range(7):
    t=0
    f=0
    n=0
    x=0
    with open(filePath[7]+"dataset_0.5.jsonl") as file1:
        for line1 in file1:
            label=line1.split("label: ")[-1]
            ref1=line1.split("Response: ")[0].split("Reference: ")
            if len(ref1)==2:
                ref1=ref1[1]
            else:
                ref1=ref1[1]+"Reference: "+ref1[2]
            pre=n
            flag=False
            ref1=ref1[:-8]
            with open(filePath[7]+"detections_"+str(i+1)+".jsonl") as file:
                mark=[]
                for line in file:
                    answer=line.split("Result: ")[1]
                    ref=line.split("P2: ")[0].split("P1: ")[1]
                    ref=ref.encode().decode('unicode_escape')
                    if i==6:
                        ref=ref[1:-15]
                    else:
                        ref=ref[1:-8]
                    if ref.strip()==ref1.strip():
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                tlist[i].append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                flist[i].append(ref1)
                    if flag==False:
                        # 
                        c1=ref.strip().split(",")[0]
                        c2=ref1.strip().split(",")[0]
                        if c1.strip()==c2.strip():
                            flag=True
                            n+=1
                            mark.append(',')
                            if 'true' in label: #hallucination
                                x+=1
                                if identifyConflict(answer): 
                                    t+=1
                                else:    
                                    tlist[i].append(ref1)
                            else: # factual
                                if identifyConflict(answer): 
                                    f+=1
                                else:    
                                    flist[i].append(ref1)
                    if flag==False:
                        # 
                        c1=ref.strip().split(".")
                        c2=ref1.strip().split(".")
                        if len(c1)>2 and len(c2)>2 and c1[-2]==c2[-2]:
                            flag=True
                            n+=1
                            mark.append('.')
                            if 'true' in label: #hallucination
                                x+=1
                                if identifyConflict(answer): 
                                    t+=1
                                else:    
                                    tlist[i].append(ref1)
                            else: # factual
                                if identifyConflict(answer): 
                                    f+=1
                                else:    
                                    flist[i].append(ref1)
                    if flag==False:
                        # 
                        c1=ref.strip().split("\\\\\\\\n")
                        c2=ref1.strip().split("\\\\\\\\n")
                        if len(c1)>2 and len(c2)>2 and c1[-1]==c2[-1]:
                            flag=True
                            mark.append('n')
                            n+=1
                            if 'true' in label: #hallucination
                                x+=1
                                if identifyConflict(answer): 
                                    t+=1
                                else:    
                                    tlist[i].append(ref1)
                            else: # factual
                                if identifyConflict(answer): 
                                    f+=1
                                else:    
                                    flist[i].append(ref1)
                    if flag==False:
                        # 
                        c1=ref.strip().split("\\\\\\\\\\\\\\\"")
                        if i==6:
                            c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                        c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                        if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                            flag=True
                            mark.append('\"')
                            n+=1
                            if 'true' in label: #hallucination
                                if identifyConflict(answer):
                                    t+=1
                                else:    
                                    tlist[i].append(ref1)
                            else: # factual
                                x+=1
                                if identifyConflict(answer):
                                    f+=1
                                else:    
                                    flist[i].append(ref1)
                    if flag==False:
                        if (" Klobuchar and her husband John Bessler have two daughters," in ref and " Klobuchar and her husband John Bessler have two daughters," in ref1) or ("According to an article published in Forbes on August 21, 2018, titled" in ref and "According to an article published in Forbes on August 21, 2018, titled" in ref1) or ("The race was set to take place in October 2016 at the Central Harbour Front Event Space," in ref and "The race was set to take place in October 2016 at the Central Harbour Front Event Space," in ref1):
                            flag=True
                            mark.append('k')
                            n+=1
                            if 'true' in label: #hallucination
                                if identifyConflict(answer):
                                    t+=1
                                else:    
                                    tlist[i].append(ref1)
                            else: # factual
                                x+=1
                                if identifyConflict(answer):
                                    f+=1
                                else:    
                                    flist[i].append(ref1)
            if n-pre!=1:
                print(mark)
                print(n-pre)
                print(ref1)
                print()
    print(x)
    print(t)
    print(num[-1]-t,(num[-1]-t)/num[-1])
    print(f,f/num[-1])
    print('matched num=',n)

239
200
39 0.16317991631799164
204 0.8535564853556485
matched num= 478
239
150
89 0.3723849372384937
174 0.7280334728033473
matched num= 478
239
156
83 0.3472803347280335
164 0.6861924686192469
matched num= 478
239
191
48 0.200836820083682
193 0.8075313807531381
matched num= 478
239
143
96 0.401673640167364
148 0.6192468619246861
matched num= 478
239
181
58 0.24267782426778242
200 0.8368200836820083
matched num= 478
238
202
37 0.15481171548117154
204 0.8535564853556485
matched num= 478


In [286]:
for i in range(6):
    t=0
    f=0
    n=0
    x=0
    with open(filePath[7]+"dataset_0.5.jsonl") as file1:
        for line1 in file1:
            label=line1.split("label: ")[-1]
            ref1=line1.split("Response: ")[0].split("Reference: ")
            if len(ref1)==2:
                ref1=ref1[1]
            else:
                ref1=ref1[1]+"Reference: "+ref1[2]
            pre=n
            flag=False
            ref1=ref1[:-8]
            with open(filePath[7]+"detections_"+str(i+1)+"_1.jsonl") as file:
                mark=[]
                for line in file:
                    answer=line.split("Result: ")[1]
                    ref=line.split("P2: ")[0].split("P1: ")[1]
                    ref=ref.encode().decode('unicode_escape')
                    if i==6:
                        ref=ref[1:-15]
                    else:
                        ref=ref[1:-8]
                    if ref.strip()==ref1.strip():
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                tlist[i+7].append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                flist[i+7].append(ref1)
                    if flag==False:
                        # 
                        c1=ref.strip().split(",")[0]
                        c2=ref1.strip().split(",")[0]
                        if c1.strip()==c2.strip():
                            flag=True
                            n+=1
                            mark.append(',')
                            if 'true' in label: #hallucination
                                x+=1
                                if identifyConflict(answer): 
                                    t+=1
                                else:    
                                    tlist[i+7].append(ref1)
                            else: # factual
                                if identifyConflict(answer): 
                                    f+=1
                                else:    
                                    flist[i+7].append(ref1)
                    if flag==False:
                        # 
                        c1=ref.strip().split(".")
                        c2=ref1.strip().split(".")
                        if len(c1)>2 and len(c2)>2 and c1[-2]==c2[-2]:
                            flag=True
                            n+=1
                            mark.append('.')
                            if 'true' in label: #hallucination
                                x+=1
                                if identifyConflict(answer): 
                                    t+=1
                                else:    
                                    tlist[i+7].append(ref1)
                            else: # factual
                                if identifyConflict(answer): 
                                    f+=1
                                else:    
                                    flist[i+7].append(ref1)
                    if flag==False:
                        # 
                        c1=ref.strip().split("\\\\\\\\n")
                        c2=ref1.strip().split("\\\\\\\\n")
                        if len(c1)>2 and len(c2)>2 and c1[-1]==c2[-1]:
                            flag=True
                            mark.append('n')
                            n+=1
                            if 'true' in label: #hallucination
                                x+=1
                                if identifyConflict(answer): 
                                    t+=1
                                else:    
                                    tlist[i+7].append(ref1)
                            else: # factual
                                if identifyConflict(answer): 
                                    f+=1
                                else:    
                                    flist[i+7].append(ref1)
                    if flag==False:
                        # 
                        c1=ref.strip().split("\\\\\\\\\\\\\\\"")
                        if i==6:
                            c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                        c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                        if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                            flag=True
                            mark.append('\"')
                            n+=1
                            if 'true' in label: #hallucination
                                if identifyConflict(answer):
                                    t+=1
                                else:    
                                    tlist[i+7].append(ref1)
                            else: # factual
                                x+=1
                                if identifyConflict(answer):
                                    f+=1
                                else:    
                                    flist[i+7].append(ref1)
                    if flag==False:
                        if (" Klobuchar and her husband John Bessler have two daughters," in ref and " Klobuchar and her husband John Bessler have two daughters," in ref1) or ("According to an article published in Forbes on August 21, 2018, titled" in ref and "According to an article published in Forbes on August 21, 2018, titled" in ref1) or ("The race was set to take place in October 2016 at the Central Harbour Front Event Space," in ref and "The race was set to take place in October 2016 at the Central Harbour Front Event Space," in ref1):
                            flag=True
                            mark.append('k')
                            n+=1
                            if 'true' in label: #hallucination
                                if identifyConflict(answer):
                                    t+=1
                                else:    
                                    tlist[i+7].append(ref1)
                            else: # factual
                                x+=1
                                if identifyConflict(answer):
                                    f+=1
                                else:    
                                    flist[i+7].append(ref1)
            if n-pre!=1:
                print(mark)
                print(n-pre)
                print(ref1)
                print()
    print(x)
    print(t)
    print(num[-1]-t,(num[-1]-t)/num[-1])
    print(f,f/num[-1])
    print('matched num=',n)

239
199
40 0.16736401673640167
188 0.7866108786610879
matched num= 478
239
182
57 0.2384937238493724
176 0.7364016736401674
matched num= 478
239
194
45 0.18828451882845187
206 0.8619246861924686
matched num= 478
239
154
85 0.35564853556485354
170 0.7112970711297071
matched num= 478
239
149
90 0.37656903765690375
153 0.6401673640167364
matched num= 478
239
111
128 0.5355648535564853
114 0.4769874476987448
matched num= 478


In [295]:
minV=0
for kk in range(13-7):
    for jj in range(kk+1,13-6):
        for ii in range(jj+1,13-5):
            for i in range(ii+1,13-4):
                for j in range(i+1,13-3):
                     for k in range(j+1,13-2):
                        for m in range(k+1,13-1):
                            for n in range(m+1,13-0):
                                #for q in range(n+1,13-0):
                                    #for w in range(q+1,13-0):
                                        #for z in range(w+1,13-0):
                                            #for y in range(z+1,13-0):
                                                a=tlist[kk]+tlist[jj]+tlist[ii]+tlist[i]+tlist[j]+tlist[k]+tlist[m]+tlist[n]#+tlist[q]+tlist[w]+tlist[z]+tlist[y]+tlist[p] 
                                                a=list(set(a))
                                                aa=len(a)
                                                a=flist[kk]+flist[jj]+flist[ii]+flist[i]+flist[j]+flist[k]+flist[m]+flist[n]#+flist[q]+flist[w]+flist[z]+flist[y]+flist[p] 
                                                a=list(set(a))
                                                bb=len(a)
                                                TN=(num[-1]-bb)/(num[-1]*2)
                                                TP=aa/(num[-1]*2)
                                                FN=(num[-1]-aa)/(num[-1]*2)
                                                FP=bb/(num[-1]*2)
                                                F1=2*TP/(2*TP+FP+FN)
                                                if TN+TP>=minV:
                                                    print(kk,jj,ii,i)#,j,k,m,n,q,w,z,y,p)
                                                    print(TN+TP)
                                                    print(F1)
                                                    minV=TN+TP

0 1 2 3
0.5146443514644352
0.639751552795031
0 1 2 3
0.5167364016736402
0.640746500777605
0 1 2 3
0.5313807531380753
0.6521739130434782
0 1 2 3
0.5334728033472803
0.661608497723824
0 1 2 3
0.5376569037656904
0.6646433990895296
0 1 2 3
0.5418410041841004
0.6551181102362205
0 1 2 3
0.5418410041841004
0.6572769953051643


# wice+0.1

In [224]:
tlist=[[] for _ in range(13)]
flist=[[] for _ in range(13)]

In [320]:
for i in range(7):
    t=0
    f=0
    n=0
    x=0
    with open(filePath[8]+"dataset_0.1.jsonl") as file1:
        for line1 in file1:
            label=line1.split("label: ")[-1]
            ref1=line1.split("Response: ")[0].split("Reference: ")
            if len(ref1)==2:
                ref1=ref1[1]
            else:
                ref1=ref1[1]+"Reference: "+ref1[2]
            pre=n
            flag=False
            ref1=ref1[:-8]
            with open(filePath[8]+"detections_"+str(i+1)+".jsonl") as file:
                mark=[]
                for line in file:
                    answer=line.split("Result: ")[1]
                    ref=line.split("P2: ")[0].split("P1: ")[1]
                    ref=ref.encode().decode('unicode_escape')
                    ref=ref[1:-8]
                    if ref.strip()==ref1.strip():
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                tlist[i].append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                flist[i].append(ref1)
                    if flag==False:
                        # 
                        c1=ref.strip().split(",")[0]
                        c2=ref1.strip().split(",")[0]
                        if c1.strip()==c2.strip():
                            flag=True
                            n+=1
                            mark.append(',')
                            if 'true' in label: #hallucination
                                x+=1
                                if identifyConflict(answer): 
                                    t+=1
                                else:    
                                    tlist[i].append(ref1)
                            else: # factual
                                if identifyConflict(answer): 
                                    f+=1
                                else:    
                                    flist[i].append(ref1)
                    if flag==False:
                        # 
                        c1=ref.strip().split(".")
                        c2=ref1.strip().split(".")
                        if len(c1)>2 and len(c2)>2 and c1[-2]==c2[-2]:
                            flag=True
                            n+=1
                            mark.append('.')
                            if 'true' in label: #hallucination
                                x+=1
                                if identifyConflict(answer): 
                                    t+=1
                                else:    
                                    tlist[i].append(ref1)
                            else: # factual
                                if identifyConflict(answer): 
                                    f+=1
                                else:    
                                    flist[i].append(ref1)
                    if flag==False:
                        # 
                        c1=ref.strip().split("\\\\n")
                        c2=ref1.strip().split("\\\\n")
                        if len(c1)>2 and len(c2)>2 and c1[-1]==c2[-1]:
                            flag=True
                            mark.append('n')
                            n+=1
                            if 'true' in label: #hallucination
                                x+=1
                                if identifyConflict(answer): 
                                    t+=1
                                else:    
                                    tlist[i].append(ref1)
                            else: # factual
                                if identifyConflict(answer): 
                                    f+=1
                                else:    
                                    flist[i].append(ref1)
                    if flag==False:
                        # 
                        c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                        c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                        if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                            flag=True
                            mark.append('\"')
                            n+=1
                            if 'true' in label: #hallucination
                                if identifyConflict(answer):
                                    t+=1
                                else:    
                                    tlist[i].append(ref1)
                            else: # factual
                                x+=1
                                if identifyConflict(answer):
                                    f+=1
                                else:    
                                    flist[i].append(ref1)
            if n-pre!=1:
                print(mark)
                print(n-pre)
                print(ref1)
                print()
    print(x)
    print(t)
    print(num[-1]-t,(num[-1]-t)/num[-1])
    print(f,f/num[-1])
    print('matched num=',n)

242
214
31 0.12653061224489795
221 0.9020408163265307
matched num= 484
242
181
64 0.2612244897959184
191 0.7795918367346939
matched num= 484
242
167
78 0.3183673469387755
178 0.726530612244898
matched num= 484
242
199
46 0.18775510204081633
217 0.8857142857142857
matched num= 484
242
177
68 0.27755102040816326
174 0.710204081632653
matched num= 484
242
198
47 0.19183673469387755
210 0.8571428571428571
matched num= 484
242
189
56 0.22857142857142856
179 0.7306122448979592
matched num= 484


In [226]:
for i in range(6):
    t=0
    f=0
    n=0
    x=0
    with open(filePath[8]+"dataset_0.1.jsonl") as file1:
        for line1 in file1:
            label=line1.split("label: ")[-1]
            ref1=line1.split("Response: ")[0].split("Reference: ")
            if len(ref1)==2:
                ref1=ref1[1]
            else:
                ref1=ref1[1]+"Reference: "+ref1[2]
            pre=n
            flag=False
            ref1=ref1[:-8]
            with open(filePath[8]+"detections_"+str(i+1)+"_1.jsonl") as file:
                mark=[]
                for line in file:
                    answer=line.split("Result: ")[1]
                    ref=line.split("P2: ")[0].split("P1: ")[1]
                    ref=ref.encode().decode('unicode_escape')
                    ref=ref[1:-8]
                    if ref.strip()==ref1.strip():
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                tlist[i+7].append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                flist[i+7].append(ref1)
                    if flag==False:
                        # 
                        c1=ref.strip().split(",")[0]
                        c2=ref1.strip().split(",")[0]
                        if c1.strip()==c2.strip():
                            flag=True
                            n+=1
                            mark.append(',')
                            if 'true' in label: #hallucination
                                x+=1
                                if identifyConflict(answer): 
                                    t+=1
                                else:    
                                    tlist[i+7].append(ref1)
                            else: # factual
                                if identifyConflict(answer): 
                                    f+=1
                                else:    
                                    flist[i+7].append(ref1)
                    if flag==False:
                        # 
                        c1=ref.strip().split(".")
                        c2=ref1.strip().split(".")
                        if len(c1)>2 and len(c2)>2 and c1[-2]==c2[-2]:
                            flag=True
                            n+=1
                            mark.append('.')
                            if 'true' in label: #hallucination
                                x+=1
                                if identifyConflict(answer): 
                                    t+=1
                                else:    
                                    tlist[i+7].append(ref1)
                            else: # factual
                                if identifyConflict(answer): 
                                    f+=1
                                else:    
                                    flist[i+7].append(ref1)
                    if flag==False:
                        # 
                        c1=ref.strip().split("\\\\n")
                        c2=ref1.strip().split("\\\\n")
                        if len(c1)>2 and len(c2)>2 and c1[-1]==c2[-1]:
                            flag=True
                            mark.append('n')
                            n+=1
                            if 'true' in label: #hallucination
                                x+=1
                                if identifyConflict(answer): 
                                    t+=1
                                else:    
                                    tlist[i+7].append(ref1)
                            else: # factual
                                if identifyConflict(answer): 
                                    f+=1
                                else:    
                                    flist[i+7].append(ref1)
                    if flag==False:
                        # 
                        c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                        c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                        if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                            flag=True
                            mark.append('\"')
                            n+=1
                            if 'true' in label: #hallucination
                                if identifyConflict(answer):
                                    t+=1
                                else:    
                                    tlist[i+7].append(ref1)
                            else: # factual
                                x+=1
                                if identifyConflict(answer):
                                    f+=1
                                else:    
                                    flist[i+7].append(ref1)
                    if flag==False:
                        if "cording to Deadline, in February 2019, it was announced that Armie Hammer, Gary Old" in ref and "cording to Deadline, in February 2019, it was announced that Armie Hammer, Gary Old" in ref1:
                            flag=True
                            mark.append('k')
                            n+=1
                            if 'true' in label: #hallucination
                                if identifyConflict(answer):
                                    t+=1
                                else:    
                                    tlist[i+7].append(ref1)
                            else: # factual
                                x+=1
                                if identifyConflict(answer):
                                    f+=1
                                else:    
                                    flist[i+7].append(ref1)
                                                      
            if n-pre!=1:
                print(mark)
                print(n-pre)
                print(ref1)
                print()
    print(x)
    print(t)
    print(num[-1]-t,(num[-1]-t)/num[-1])
    print(f,f/num[-1])
    print('matched num=',n)

242
220
22 0.09090909090909091
222 0.9173553719008265
matched num= 484
242
198
44 0.18181818181818182
202 0.8347107438016529
matched num= 484
242
217
25 0.10330578512396695
217 0.8966942148760331
matched num= 484
242
171
71 0.29338842975206614
187 0.7727272727272727
matched num= 484
242
141
101 0.41735537190082644
154 0.6363636363636364
matched num= 484
255
139
103 0.4256198347107438
147 0.6074380165289256
matched num= 484


In [236]:
minV=0
for kk in range(13-8):
    for jj in range(kk+1,13-7):
        for ii in range(jj+1,13-6):
            for i in range(ii+1,13-5):
                for j in range(i+1,13-4):
                     for k in range(j+1,13-3):
                        for m in range(k+1,13-2):
                            for n in range(m+1,13-1):
                                for q in range(n+1,13-0):
                                    #for w in range(q+1,13-0):
                                        #for z in range(w+1,13-0):
                                            #for y in range(z+1,13-0):
                                                a=tlist[kk]+tlist[jj]+tlist[ii]+tlist[i]+tlist[j]+tlist[k]+tlist[m]+tlist[n]+tlist[q]#+tlist[w]+tlist[z]+tlist[y]+tlist[p] 
                                                a=list(set(a))
                                                aa=len(a)
                                                a=flist[kk]+flist[jj]+flist[ii]+flist[i]+flist[j]+flist[k]+flist[m]+flist[n]+flist[q]#+flist[w]+flist[z]+flist[y]+flist[p] 
                                                a=list(set(a))
                                                bb=len(a)
                                                TN=(num[-1]-bb)/(num[-1]*2)
                                                TP=aa/(num[-1]*2)
                                                FN=(num[-1]-aa)/(num[-1]*2)
                                                FP=bb/(num[-1]*2)
                                                F1=2*TP/(2*TP+FP+FN)
                                                if TN+TP>=minV:
                                                    print(kk,jj,ii,i)#,j,k,m,n,q,w,z,y,p)
                                                    print(TN+TP)
                                                    print(F1)
                                                    minV=TN+TP

0 1 2 3
0.5082644628099173
0.6006711409395973
0 1 2 3
0.5082644628099173
0.5966101694915255
0 1 2 3
0.5227272727272727
0.6231647634584013
0 1 2 3
0.5227272727272727
0.6268174474959612
0 1 2 3
0.5289256198347108
0.6095890410958904
0 1 2 3
0.5454545454545454
0.6381578947368421
0 1 3 4
0.5495867768595041
0.6241379310344827


# climate+0.1

In [335]:
tlist=[[] for _ in range(13)]
flist=[[] for _ in range(13)]

In [336]:
for i in range(7):
    t=0
    f=0
    n=0
    x=0
    with open(filePath[2]+"dataset_0.1.jsonl") as file1:
        for line1 in file1:
            label=line1.split("label: ")[-1]
            ref1=line1.split("Response: ")[0].split("Reference: ")
            if len(ref1)==2:
                ref1=ref1[1]
            else:
                ref1=ref1[1]+"Reference: "+ref1[2]
            pre=n
            flag=False
            ref1=ref1[:-8]
            with open(filePath[2]+"detections_"+str(i+1)+".jsonl") as file:
                mark=[]
                for line in file:
                    answer=line.split("Result: ")[1]
                    ref=line.split("P2: ")[0].split("P1: ")[1]
                    ref=ref.encode().decode('unicode_escape')
                    ref=ref[1:-15]
                    if ref.strip()==ref1.strip():
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                tlist[i].append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                flist[i].append(ref1)
#                     if flag==False:
#                         # 
#                         c1=ref.strip().split(",")[0]
#                         c2=ref1.strip().split(",")[0]
#                         if c1.strip()==c2.strip():
#                             flag=True
#                             n+=1
#                             mark.append(',')
#                             if 'true' in label: #hallucination
#                                 x+=1
#                                 if identifyConflict(answer): 
#                                     t+=1
#                                 else:    
#                                     tlist[i].append(ref1)
#                             else: # factual
#                                 if identifyConflict(answer): 
#                                     f+=1
#                                 else:    
#                                     flist[i].append(ref1)
                    if flag==False:
                        # 
                        c1=ref.strip().split(".")
                        c2=ref1.strip().split(".")
                        if len(c1)>2 and len(c2)>2 and c1[-2]==c2[-2]:
                            flag=True
                            n+=1
                            mark.append('.')
                            if 'true' in label: #hallucination
                                x+=1
                                if identifyConflict(answer): 
                                    t+=1
                                else:    
                                    tlist[i].append(ref1)
                            else: # factual
                                if identifyConflict(answer): 
                                    f+=1
                                else:    
                                    flist[i].append(ref1)
                    if flag==False:
                        # 
                        c1=ref.strip().split("\\\\n")
                        c2=ref1.strip().split("\\\\n")
                        if len(c1)>2 and len(c2)>2 and c1[-1]==c2[-1]:
                            flag=True
                            mark.append('n')
                            n+=1
                            if 'true' in label: #hallucination
                                x+=1
                                if identifyConflict(answer): 
                                    t+=1
                                else:    
                                    tlist[i].append(ref1)
                            else: # factual
                                if identifyConflict(answer): 
                                    f+=1
                                else:    
                                    flist[i].append(ref1)
                    if flag==False:
                        # 
                        c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                        c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                        if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                            flag=True
                            mark.append('\"')
                            n+=1
                            if 'true' in label: #hallucination
                                if identifyConflict(answer):
                                    t+=1
                                else:    
                                    tlist[i].append(ref1)
                            else: # factual
                                x+=1
                                if identifyConflict(answer):
                                    f+=1
                                else:    
                                    flist[i].append(ref1)
                    if flag==False:
                        if "2019-07-03-climate-declaration_en.pdf Summarize the refer" in ref and "2019-07-03-climate-declaration_en.pdf Summarize the refer"  in ref1:
                            flag=True
                            n+=1
                            if 'true' in label: #hallucination
                                if identifyConflict(answer):
                                    t+=1
                                else:    
                                    tlist[i].append(ref1)
                            else: # factual
                                x+=1
                                if identifyConflict(answer):
                                    f+=1
                                else:    
                                    flist[i].append(ref1)
            if n-pre!=1:
                print(mark)
                print(n-pre)
                print(ref1)
                print()
    print(x)
    print(t)
    print(num[-1]-t,(num[-1]-t)/num[-1])
    print(f,f/num[-1])
    print('matched num=',n)

181
154
91 0.37142857142857144
159 0.6489795918367347
matched num= 350
181
142
103 0.4204081632653061
153 0.6244897959183674
matched num= 350
181
144
101 0.4122448979591837
156 0.636734693877551
matched num= 350
181
153
92 0.37551020408163266
141 0.5755102040816327
matched num= 350
181
151
94 0.3836734693877551
145 0.5918367346938775
matched num= 350
181
149
96 0.39183673469387753
148 0.6040816326530613
matched num= 350
181
140
105 0.42857142857142855
143 0.5836734693877551
matched num= 350


In [337]:
for i in range(6):
    t=0
    f=0
    n=0
    x=0
    with open(filePath[2]+"dataset_0.1.jsonl") as file1:
        for line1 in file1:
            label=line1.split("label: ")[-1]
            ref1=line1.split("Response: ")[0].split("Reference: ")
            if len(ref1)==2:
                ref1=ref1[1]
            else:
                ref1=ref1[1]+"Reference: "+ref1[2]
            pre=n
            flag=False
            ref1=ref1[:-8]
            with open(filePath[2]+"detections_"+str(i+1)+"_1.jsonl") as file:
                mark=[]
                for line in file:
                    answer=line.split("Result: ")[1]
                    ref=line.split("P2: ")[0].split("P1: ")[1]
                    ref=ref.encode().decode('unicode_escape')
                    ref=ref[1:-15]
                    if ref.strip()==ref1.strip():
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                tlist[i+7].append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                flist[i+7].append(ref1)
#                     if flag==False:
#                         # 
#                         c1=ref.strip().split(",")[0]
#                         c2=ref1.strip().split(",")[0]
#                         if c1.strip()==c2.strip():
#                             flag=True
#                             n+=1
#                             mark.append(',')
#                             if 'true' in label: #hallucination
#                                 x+=1
#                                 if identifyConflict(answer): 
#                                     t+=1
#                                 else:    
#                                     tlist[i].append(ref1)
#                             else: # factual
#                                 if identifyConflict(answer): 
#                                     f+=1
#                                 else:    
#                                     flist[i].append(ref1)
                    if flag==False:
                        # 
                        c1=ref.strip().split(".")
                        c2=ref1.strip().split(".")
                        if len(c1)>2 and len(c2)>2 and c1[-2]==c2[-2]:
                            flag=True
                            n+=1
                            mark.append('.')
                            if 'true' in label: #hallucination
                                x+=1
                                if identifyConflict(answer): 
                                    t+=1
                                else:    
                                    tlist[i+7].append(ref1)
                            else: # factual
                                if identifyConflict(answer): 
                                    f+=1
                                else:    
                                    flist[i+7].append(ref1)
                    if flag==False:
                        # 
                        c1=ref.strip().split("\\\\n")
                        c2=ref1.strip().split("\\\\n")
                        if len(c1)>2 and len(c2)>2 and c1[-1]==c2[-1]:
                            flag=True
                            mark.append('n')
                            n+=1
                            if 'true' in label: #hallucination
                                x+=1
                                if identifyConflict(answer): 
                                    t+=1
                                else:    
                                    tlist[i+7].append(ref1)
                            else: # factual
                                if identifyConflict(answer): 
                                    f+=1
                                else:    
                                    flist[i+7].append(ref1)
                    if flag==False:
                        # 
                        c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                        c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                        if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                            flag=True
                            mark.append('\"')
                            n+=1
                            if 'true' in label: #hallucination
                                if identifyConflict(answer):
                                    t+=1
                                else:    
                                    tlist[i+7].append(ref1)
                            else: # factual
                                x+=1
                                if identifyConflict(answer):
                                    f+=1
                                else:    
                                    flist[i+7].append(ref1)
                    if flag==False:
                        if "2019-07-03-climate-declaration_en.pdf Summarize the refer" in ref and "2019-07-03-climate-declaration_en.pdf Summarize the refer"  in ref1:
                            flag=True
                            n+=1
                            if 'true' in label: #hallucination
                                if identifyConflict(answer):
                                    t+=1
                                else:    
                                    tlist[i+7].append(ref1)
                            else: # factual
                                x+=1
                                if identifyConflict(answer):
                                    f+=1
                                else:    
                                    flist[i+7].append(ref1)
            if n-pre!=1:
                print(mark)
                print(n-pre)
                print(ref1)
                print()
    print(x)
    print(t)
    print(num[-1]-t,(num[-1]-t)/num[-1])
    print(f,f/num[-1])
    print('matched num=',n)

181
145
100 0.40816326530612246
153 0.6244897959183674
matched num= 350
181
145
100 0.40816326530612246
141 0.5755102040816327
matched num= 350
181
145
100 0.40816326530612246
158 0.6448979591836734
matched num= 350
181
145
100 0.40816326530612246
149 0.6081632653061224
matched num= 350
181
129
116 0.47346938775510206
137 0.5591836734693878
matched num= 350
181
133
112 0.45714285714285713
148 0.6040816326530613
matched num= 350


In [346]:
minV=0
for kk in range(13-7):
    for jj in range(kk+1,13-6):
        for ii in range(jj+1,13-5):
            for i in range(ii+1,13-4):
                for j in range(i+1,13-3):
                     for k in range(j+1,13-2):
                        for m in range(k+1,13-1):
                            for n in range(m+1,13-0):
                                #for q in range(n+1,13-0):
                                    #for w in range(q+1,13-0):
                                        #for z in range(w+1,13-0):
                                            #for y in range(z+1,13-0):
                                                a=tlist[kk]+tlist[jj]+tlist[ii]+tlist[i]+tlist[j]+tlist[k]+tlist[m]+tlist[n]#+tlist[q]+tlist[w]+tlist[z]+tlist[y]+tlist[p] 
                                                a=list(set(a))
                                                aa=len(a)
                                                a=flist[kk]+flist[jj]+flist[ii]+flist[i]+flist[j]+flist[k]+flist[m]+flist[n]#+flist[q]+flist[w]+flist[z]+flist[y]+flist[p] 
                                                a=list(set(a))
                                                bb=len(a)
                                                TN=(num[-1]-bb)/(num[-1]*2)
                                                TP=aa/(num[-1]*2)
                                                FN=(num[-1]-aa)/(num[-1]*2)
                                                FP=bb/(num[-1]*2)
                                                F1=2*TP/(2*TP+FP+FN)
                                                if TN+TP>=minV:
                                                    print(kk,jj,ii,i)#,j,k,m,n,q,w,z,y,p)
                                                    print(TN+TP)
                                                    print(F1)
                                                    minV=TN+TP

0 1 2 3
0.46122448979591835
0.39449541284403666
0 1 2 3
0.47755102040816333
0.41284403669724773
0 1 2 3
0.4877551020408163
0.44097995545657015
0 1 2 3
0.4897959183673469
0.41314553990610325
0 1 2 3
0.49591836734693884
0.4449438202247191
0 1 2 3
0.5061224489795918
0.4598214285714286
0 1 2 3
0.5081632653061224
0.4559819413092551
0 1 2 3
0.5081632653061224
0.4656319290465632
0 1 2 3
0.5081632653061224
0.46325167037861914
0 1 2 3
0.5081632653061224
0.4703296703296703
0 1 2 3
0.5081632653061224
0.46325167037861914
0 1 2 3
0.5142857142857142
0.46875
0 1 2 3
0.5142857142857142
0.4780701754385965
0 1 2 3
0.5163265306122449
0.47216035634743875
0 1 2 4
0.5163265306122449
0.47450110864745015
0 1 2 4
0.5163265306122449
0.4674157303370786
0 1 2 4
0.5244897959183674
0.4716553287981859
0 1 2 5
0.5285714285714286
0.4832214765100671
0 1 2 6
0.5285714285714286
0.4900662251655629
0 2 5 6
0.5285714285714286
0.4832214765100671
0 2 5 6
0.5306122448979591
0.4866071428571429


# climate+0.5

In [596]:
t=0
f=0
t1=[]
f1=[]
n=0
with open(filePath[1]+"dataset_0.5.jsonl") as file1:
    for line1 in file1:
        label=line1.split("label: ")[-1]
        ref1=line1.split("Response: ")[0].split("Reference: ")[1]
        pre=n
        flag=False
        ref1=ref1[:-8]
        with open(filePath[1]+"detections_1.jsonl") as file:
            for line in file:
                answer=line.split("Result: ")[1]
                ref=line.split("P2: ")[0].split("P1: ")[1]
                ref=ref.encode().decode('unicode_escape')
                ref=ref[1:-15]
                if ref.strip()==ref1.strip():
                    flag=True
                    n+=1
                    if 'true' in label: #hallucination
                        if identifyConflict(answer):
                            t+=1
                        else:    
                            t1.append(ref1)
                    else: # factual
                        if identifyConflict(answer):
                            f+=1
                        else:    
                            f1.append(ref1)
#                 if flag==False:
#                     # 
#                     c1=ref.strip().split(".")[0]
#                     c2=ref1.strip().split(".")[0]
#                     if c1.strip()==c2.strip():
#                         flag=True
#                         n+=1
#                         if 'true' in label: #hallucination
#                             if identifyConflict(answer):
#                                 t+=1
#                             else:    
#                                 t11.append(ref1)
#                         else: # factual
#                             if identifyConflict(answer):
#                                 f+=1
#                             else:    
#                                 f11.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split(".")
                    c2=ref1.strip().split(".")
                    if len(c1)>2 and len(c2)>2 and c1[-2]==c2[-2]:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t1.append(ref1)
                        else: # factual
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f1.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\n")[-1]
                    c2=ref1.strip().split("\\\\n")[-1]
                    if c1==c2:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t1.append(ref1)
                        else: # factual
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f1.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                    c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                    if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t1.append(ref1)
                        else: # factual
                            x+=1
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f1.append(ref1)
                if flag==False:
                    uni=ref.strip().split('\\u')[-1][4:]
                    length=len(uni)
                    if ref1.strip()[-length:] == uni:
                        flag = True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t1.append(ref1)
                        else: # factual
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f1.append(ref1)
        if n-pre!=1:
            print(n-pre)
            print(ref1)
            print()
print(num[-1]-t,(num[-1]-t)/num[-1])
print(f,f/num[-1])
print('matched num=',n)

2
 https://www.thegwpf.com/european-climate-declaration/ The European Climate Declaration (ECD) is a document signed by over 500 scientists and professionals in climate and related fields, stating that there is no climate emergency and that the current climate policies are not based on sound science. The declaration emphasizes that the Earth's climate has been changing for centuries and that the current warming period is not unusual when considering the geological history of the Earth. It also states that the impact of human activities on the climate is relatively small compared to natural factors and that the current policies aimed at reducing CO2 emissions will have little effect on the climate. The signatories of the ECD include climatologists, physicists, chemists, biologists, geologists, and other experts from Europe and around the world. They come from universities, research institutions, and private companies, and represent a wide range of scientific disciplines.

2
 https://www

In [597]:
t=0
f=0
t11=[]
f11=[]
n=0
with open(filePath[1]+"dataset_0.5.jsonl") as file1:
    for line1 in file1:
        label=line1.split("label: ")[-1]
        ref1=line1.split("Response: ")[0].split("Reference: ")[1]
        pre=n
        flag=False
        ref1=ref1[:-8]
        with open(filePath[1]+"detections_1_1.jsonl") as file:
            for line in file:
                answer=line.split("Result: ")[1]
                ref=line.split("P2: ")[0].split("P1: ")[1]
                ref=ref.encode().decode('unicode_escape')
                ref=ref[1:-15]
                if ref.strip()==ref1.strip():
                    flag=True
                    n+=1
                    if 'true' in label: #hallucination
                        if identifyConflict(answer):
                            t+=1
                        else:    
                            t11.append(ref1)
                    else: # factual
                        if identifyConflict(answer):
                            f+=1
                        else:    
                            f11.append(ref1)
#                 if flag==False:
#                     # 
#                     c1=ref.strip().split(".")[0]
#                     c2=ref1.strip().split(".")[0]
#                     if c1.strip()==c2.strip():
#                         flag=True
#                         n+=1
#                         if 'true' in label: #hallucination
#                             if identifyConflict(answer):
#                                 t+=1
#                             else:    
#                                 t11.append(ref1)
#                         else: # factual
#                             if identifyConflict(answer):
#                                 f+=1
#                             else:    
#                                 f11.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split(".")
                    c2=ref1.strip().split(".")
                    if len(c1)>2 and len(c2)>2 and c1[-2]==c2[-2]:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t11.append(ref1)
                        else: # factual
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f11.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\n")[-1]
                    c2=ref1.strip().split("\\\\n")[-1]
                    if c1==c2:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t11.append(ref1)
                        else: # factual
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f11.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                    c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                    if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t11.append(ref1)
                        else: # factual
                            x+=1
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f11.append(ref1)
                if flag==False:
                    uni=ref.strip().split('\\u')[-1][4:]
                    length=len(uni)
                    if ref1.strip()[-length:] == uni:
                        flag = True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t11.append(ref1)
                        else: # factual
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f11.append(ref1)
        if n-pre!=1:
            print(n-pre)
            print(ref1)
            print()
print(num[-1]-t,(num[-1]-t)/num[-1])
print(f,f/num[-1])
print('matched num=',n)

2
 https://www.thegwpf.com/european-climate-declaration/ The European Climate Declaration (ECD) is a document signed by over 500 scientists and professionals in climate and related fields, stating that there is no climate emergency and that the current climate policies are not based on sound science. The declaration emphasizes that the Earth's climate has been changing for centuries and that the current warming period is not unusual when considering the geological history of the Earth. It also states that the impact of human activities on the climate is relatively small compared to natural factors and that the current policies aimed at reducing CO2 emissions will have little effect on the climate. The signatories of the ECD include climatologists, physicists, chemists, biologists, geologists, and other experts from Europe and around the world. They come from universities, research institutions, and private companies, and represent a wide range of scientific disciplines.

2
 https://www

In [598]:
t=0
f=0
t2=[]
f2=[]
n=0
with open(filePath[1]+"dataset_0.5.jsonl") as file1:
    for line1 in file1:
        label=line1.split("label: ")[-1]
        ref1=line1.split("Response: ")[0].split("Reference: ")[1]
        pre=n
        flag=False
        ref1=ref1[:-8]
        with open(filePath[1]+"detections_2.jsonl") as file:
            for line in file:
                answer=line.split("Result: ")[1]
                ref=line.split("P2: ")[0].split("P1: ")[1]
                ref=ref.encode().decode('unicode_escape')
                ref=ref[1:-15]
                if ref.strip()==ref1.strip():
                    flag=True
                    n+=1
                    if 'true' in label: #hallucination
                        if identifyConflict(answer):
                            t+=1
                        else:    
                            t2.append(ref1)
                    else: # factual
                        if identifyConflict(answer):
                            f+=1
                        else:    
                            f2.append(ref1)
#                 if flag==False:
#                     # 
#                     c1=ref.strip().split(".")[0]
#                     c2=ref1.strip().split(".")[0]
#                     if c1.strip()==c2.strip():
#                         flag=True
#                         n+=1
#                         if 'true' in label: #hallucination
#                             if identifyConflict(answer):
#                                 t+=1
#                             else:    
#                                 t2.append(ref1)
#                         else: # factual
#                             if identifyConflict(answer):
#                                 f+=1
#                             else:    
#                                 f2.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split(".")
                    c2=ref1.strip().split(".")
                    if len(c1)>2 and len(c2)>2 and c1[-2]==c2[-2]:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t2.append(ref1)
                        else: # factual
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f2.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\n")[-1]
                    c2=ref1.strip().split("\\\\n")[-1]
                    if c1==c2:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t2.append(ref1)
                        else: # factual
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f2.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                    c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                    if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t2.append(ref1)
                        else: # factual
                            x+=1
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f2.append(ref1)
                if flag==False:
                    uni=ref.strip().split('\\u')[-1][4:]
                    length=len(uni)
                    if ref1.strip()[-length:] == uni:
                        flag = True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer): 
                                t+=1
                            else:    
                                t2.append(ref1)
                        else: # factual
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f2.append(ref1)
        if n-pre!=1:
            print(n-pre)
            print(ref1)
            print()
print(num[-1]-t,(num[-1]-t)/num[-1])
print(f,f/num[-1])
print('matched num=',n)

2
 https://www.thegwpf.com/european-climate-declaration/ The European Climate Declaration (ECD) is a document signed by over 500 scientists and professionals in climate and related fields, stating that there is no climate emergency and that the current climate policies are not based on sound science. The declaration emphasizes that the Earth's climate has been changing for centuries and that the current warming period is not unusual when considering the geological history of the Earth. It also states that the impact of human activities on the climate is relatively small compared to natural factors and that the current policies aimed at reducing CO2 emissions will have little effect on the climate. The signatories of the ECD include climatologists, physicists, chemists, biologists, geologists, and other experts from Europe and around the world. They come from universities, research institutions, and private companies, and represent a wide range of scientific disciplines.

2
 https://www

In [599]:
t=0
f=0
t22=[]
f22=[]
n=0
with open(filePath[1]+"dataset_0.5.jsonl") as file1:
    for line1 in file1:
        label=line1.split("label: ")[-1]
        ref1=line1.split("Response: ")[0].split("Reference: ")[1]
        pre=n
        flag=False
        ref1=ref1[:-8]
        with open(filePath[1]+"detections_2_1.jsonl") as file:
            for line in file:
                answer=line.split("Result: ")[1]
                ref=line.split("P2: ")[0].split("P1: ")[1]
                ref=ref.encode().decode('unicode_escape')
                ref=ref[1:-15]
                if ref.strip()==ref1.strip():
                    flag=True
                    n+=1
                    if 'true' in label: #hallucination
                        if identifyConflict(answer):
                            t+=1
                        else:    
                            t22.append(ref1)
                    else: # factual
                        if identifyConflict(answer):
                            f+=1
                        else:    
                            f22.append(ref1)
#                 if flag==False:
#                     # 
#                     c1=ref.strip().split(".")[0]
#                     c2=ref1.strip().split(".")[0]
#                     if c1.strip()==c2.strip():
#                         flag=True
#                         n+=1
#                         if 'true' in label: #hallucination
#                             if identifyConflict(answer):
#                                 t+=1
#                             else:    
#                                 t22.append(ref1)
#                         else: # factual
#                             if identifyConflict(answer):
#                                 f+=1
#                             else:    
#                                 f22.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split(".")
                    c2=ref1.strip().split(".")
                    if len(c1)>2 and len(c2)>2 and c1[-2]==c2[-2]:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t22.append(ref1)
                        else: # factual
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f22.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\n")[-1]
                    c2=ref1.strip().split("\\\\n")[-1]
                    if c1==c2:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t22.append(ref1)
                        else: # factual
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f22.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                    c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                    if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t22.append(ref1)
                        else: # factual
                            x+=1
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f22.append(ref1)
                if flag==False:
                    uni=ref.strip().split('\\u')[-1][4:]
                    length=len(uni)
                    if ref1.strip()[-length:] == uni:
                        flag = True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t22.append(ref1)
                        else: # factual
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f22.append(ref1)
        if n-pre!=1:
            print(ref1)
            print()
print(num[-1]-t,(num[-1]-t)/num[-1])
print(f,f/num[-1])
print('matched num=',n)

 https://www.thegwpf.com/european-climate-declaration/ The European Climate Declaration (ECD) is a document signed by over 500 scientists and professionals in climate and related fields, stating that there is no climate emergency and that the current climate policies are not based on sound science. The declaration emphasizes that the Earth's climate has been changing for centuries and that the current warming period is not unusual when considering the geological history of the Earth. It also states that the impact of human activities on the climate is relatively small compared to natural factors and that the current policies aimed at reducing CO2 emissions will have little effect on the climate. The signatories of the ECD include climatologists, physicists, chemists, biologists, geologists, and other experts from Europe and around the world. They come from universities, research institutions, and private companies, and represent a wide range of scientific disciplines.

 https://www.the

In [600]:
t=0
f=0
t3=[]
f3=[]
n=0
with open(filePath[1]+"dataset_0.5.jsonl") as file1:
    for line1 in file1:
        label=line1.split("label: ")[-1]
        ref1=line1.split("Response: ")[0].split("Reference: ")[1]
        ref1=ref1[:-8]
        pre=n
        flag=False
        with open(filePath[1]+"detections_3.jsonl") as file:
            for line in file:
                answer=line.split("Result: ")[1]
                ref=line.split("P2: ")[0].split("P1: ")[1]
                ref=ref.encode().decode('unicode_escape')
                ref=ref[1:-15]
                if ref.strip()==ref1.strip():
                    flag=True
                    n+=1
                    if 'true' in label: #hallucination
                        if identifyConflict(answer):
                            t+=1
                        else:    
                            t3.append(ref1)
                    else: # factual
                        if identifyConflict(answer):
                            f+=1
                        else:    
                            f3.append(ref1)
#                 if flag==False:
#                     # 
#                     c1=ref.strip().split(".")[0]
#                     c2=ref1.strip().split(".")[0]
#                     if c1.strip()==c2.strip():
#                         flag=True
#                         n+=1
#                         if 'true' in label: #hallucination
#                             if identifyConflict(answer):
#                                 t+=1
#                             else:    
#                                 t3.append(ref1)
#                         else: # factual
#                             if identifyConflict(answer):
#                                 f+=1
#                             else:    
#                                 f3.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split(".")
                    c2=ref1.strip().split(".")
                    if len(c1)>2 and len(c2)>2 and c1[-2]==c2[-2]:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t3.append(ref1)
                        else: # factual
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f3.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\n")[-1]
                    c2=ref1.strip().split("\\\\n")[-1]
                    if c1==c2:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t3.append(ref1)
                        else: # factual
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f3.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                    c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                    if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t3.append(ref1)
                        else: # factual
                            x+=1
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f3.append(ref1)
                if flag==False:
                    uni=ref.strip().split('\\u')[-1][4:]
                    length=len(uni)
                    if ref1.strip()[-length:] == uni:
                        flag = True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer): 
                                t+=1
                            else:    
                                t3.append(ref1)
                        else: # factual
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f3.append(ref1)
        if n-pre!=1:
            print(ref1)
            print()
print(num[-1]-t,(num[-1]-t)/num[-1])
print(f,f/num[-1])
print('matched num=',n)

 https://www.thegwpf.com/european-climate-declaration/ The European Climate Declaration (ECD) is a document signed by over 500 scientists and professionals in climate and related fields, stating that there is no climate emergency and that the current climate policies are not based on sound science. The declaration emphasizes that the Earth's climate has been changing for centuries and that the current warming period is not unusual when considering the geological history of the Earth. It also states that the impact of human activities on the climate is relatively small compared to natural factors and that the current policies aimed at reducing CO2 emissions will have little effect on the climate. The signatories of the ECD include climatologists, physicists, chemists, biologists, geologists, and other experts from Europe and around the world. They come from universities, research institutions, and private companies, and represent a wide range of scientific disciplines.

 https://www.the

In [601]:
t=0
f=0
t33=[]
f33=[]
n=0
with open(filePath[1]+"dataset_0.5.jsonl") as file1:
    for line1 in file1:
        label=line1.split("label: ")[-1]
        ref1=line1.split("Response: ")[0].split("Reference: ")[1]
        ref1=ref1[:-8]
        pre=n
        flag=False
        with open(filePath[1]+"detections_3_1.jsonl") as file:
            for line in file:
                answer=line.split("Result: ")[1]
                ref=line.split("P2: ")[0].split("P1: ")[1]
                ref=ref.encode().decode('unicode_escape')
                ref=ref[1:-15]
                if ref.strip()==ref1.strip():
                    flag=True
                    n+=1
                    if 'true' in label: #hallucination
                        if identifyConflict(answer):
                            t+=1
                        else:    
                            t33.append(ref1)
                    else: # factual
                        if identifyConflict(answer):
                            f+=1
                        else:    
                            f33.append(ref1)
#                 if flag==False:
#                     # 
#                     c1=ref.strip().split(".")[0]
#                     c2=ref1.strip().split(".")[0]
#                     if c1.strip()==c2.strip():
#                         flag=True
#                         n+=1
#                         if 'true' in label: #hallucination
#                             if identifyConflict(answer):
#                                 t+=1
#                             else:    
#                                 t33.append(ref1)
#                         else: # factual
#                             if identifyConflict(answer):
#                                 f+=1
#                             else:    
#                                 f33.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split(".")
                    c2=ref1.strip().split(".")
                    if len(c1)>2 and len(c2)>2 and c1[-2]==c2[-2]:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t33.append(ref1)
                        else: # factual
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f33.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\n")[-1]
                    c2=ref1.strip().split("\\\\n")[-1]
                    if c1==c2:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t33.append(ref1)
                        else: # factual
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f33.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                    c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                    if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t33.append(ref1)
                        else: # factual
                            x+=1
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f33.append(ref1)
                if flag==False:
                    uni=ref.strip().split('\\u')[-1][4:]
                    length=len(uni)
                    if ref1.strip()[-length:] == uni:
                        flag = True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer): 
                                t+=1
                            else:    
                                t33.append(ref1)
                        else: # factual
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f33.append(ref1)
        if n-pre!=1:
            print(ref1)
            print()
print(num[-1]-t,(num[-1]-t)/num[-1])
print(f,f/num[-1])
print('matched num=',n)

 https://www.thegwpf.com/european-climate-declaration/ The European Climate Declaration (ECD) is a document signed by over 500 scientists and professionals in climate and related fields, stating that there is no climate emergency and that the current climate policies are not based on sound science. The declaration emphasizes that the Earth's climate has been changing for centuries and that the current warming period is not unusual when considering the geological history of the Earth. It also states that the impact of human activities on the climate is relatively small compared to natural factors and that the current policies aimed at reducing CO2 emissions will have little effect on the climate. The signatories of the ECD include climatologists, physicists, chemists, biologists, geologists, and other experts from Europe and around the world. They come from universities, research institutions, and private companies, and represent a wide range of scientific disciplines.

 https://www.the

In [602]:
t=0
f=0
t4=[]
f4=[]
n=0
with open(filePath[1]+"dataset_0.5.jsonl") as file1:
    for line1 in file1:
        label=line1.split("label: ")[-1]
        ref1=line1.split("Response: ")[0].split("Reference: ")[1]
        ref1=ref1[:-8]
        pre=n
        flag=False
        with open(filePath[1]+"detections_4.jsonl") as file:
            for line in file:
                answer=line.split("Result: ")[1]
                ref=line.split("P2: ")[0].split("P1: ")[1]
                ref=ref.encode().decode('unicode_escape')
                ref=ref[1:-15]
                if ref.strip()==ref1.strip():
                    flag=True
                    n+=1
                    if 'true' in label: #hallucination
                        if identifyConflict(answer):
                            t+=1
                        else:    
                            t4.append(ref1)
                    else: # factual
                        if identifyConflict(answer):
                            f+=1
                        else:    
                            f4.append(ref1)
#                 if flag==False:
#                     # 
#                     c1=ref.strip().split(".")[0]
#                     c2=ref1.strip().split(".")[0]
#                     if c1.strip()==c2.strip():
#                         flag=True
#                         n+=1
#                         if 'true' in label: #hallucination
#                             if identifyConflict(answer):
#                                 t+=1
#                             else:    
#                                 t4.append(ref1)
#                         else: # factual
#                             if identifyConflict(answer):
#                                 f+=1
#                             else:    
#                                 f4.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split(".")
                    c2=ref1.strip().split(".")
                    if len(c1)>2 and len(c2)>2 and c1[-2]==c2[-2]:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t4.append(ref1)
                        else: # factual
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f4.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\n")[-1]
                    c2=ref1.strip().split("\\\\n")[-1]
                    if c1==c2:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t4.append(ref1)
                        else: # factual
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f4.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                    c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                    if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t4.append(ref1)
                        else: # factual
                            x+=1
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f4.append(ref1)
                if flag==False:
                    uni=ref.strip().split('\\u')[-1][4:]
                    length=len(uni)
                    if ref1.strip()[-length:] == uni:
                        flag = True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer): 
                                t+=1
                            else:    
                                t4.append(ref1)
                        else: # factual
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f4.append(ref1)
        if n-pre!=1:
            print(ref1)
            print()
print(num[-1]-t,(num[-1]-t)/num[-1])
print(f,f/num[-1])
print('matched num=',n)

 https://www.thegwpf.com/european-climate-declaration/ The European Climate Declaration (ECD) is a document signed by over 500 scientists and professionals in climate and related fields, stating that there is no climate emergency and that the current climate policies are not based on sound science. The declaration emphasizes that the Earth's climate has been changing for centuries and that the current warming period is not unusual when considering the geological history of the Earth. It also states that the impact of human activities on the climate is relatively small compared to natural factors and that the current policies aimed at reducing CO2 emissions will have little effect on the climate. The signatories of the ECD include climatologists, physicists, chemists, biologists, geologists, and other experts from Europe and around the world. They come from universities, research institutions, and private companies, and represent a wide range of scientific disciplines.

 https://www.the

In [603]:
t=0
f=0
t44=[]
f44=[]
n=0
with open(filePath[1]+"dataset_0.5.jsonl") as file1:
    for line1 in file1:
        label=line1.split("label: ")[-1]
        ref1=line1.split("Response: ")[0].split("Reference: ")[1]
        ref1=ref1[:-8]
        pre=n
        flag=False
        with open(filePath[1]+"detections_4_1.jsonl") as file:
            for line in file:
                answer=line.split("Result: ")[1]
                ref=line.split("P2: ")[0].split("P1: ")[1]
                ref=ref.encode().decode('unicode_escape')
                ref=ref[1:-15]
                if ref.strip()==ref1.strip():
                    flag=True
                    n+=1
                    if 'true' in label: #hallucination
                        if identifyConflict(answer):
                            t+=1
                        else:    
                            t44.append(ref1)
                    else: # factual
                        if identifyConflict(answer):
                            f+=1
                        else:    
                            f44.append(ref1)
#                 if flag==False:
#                     # 
#                     c1=ref.strip().split(".")[0]
#                     c2=ref1.strip().split(".")[0]
#                     if c1.strip()==c2.strip():
#                         flag=True
#                         n+=1
#                         if 'true' in label: #hallucination
#                             if identifyConflict(answer):
#                                 t+=1
#                             else:    
#                                 t44.append(ref1)
#                         else: # factual
#                             if identifyConflict(answer):
#                                 f+=1
#                             else:    
#                                 f44.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split(".")
                    c2=ref1.strip().split(".")
                    if len(c1)>2 and len(c2)>2 and c1[-2]==c2[-2]:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t44.append(ref1)
                        else: # factual
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f44.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\n")[-1]
                    c2=ref1.strip().split("\\\\n")[-1]
                    if c1==c2:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t44.append(ref1)
                        else: # factual
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f44.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                    c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                    if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t44.append(ref1)
                        else: # factual
                            x+=1
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f44.append(ref1)
                if flag==False:
                    uni=ref.strip().split('\\u')[-1][4:]
                    length=len(uni)
                    if ref1.strip()[-length:] == uni:
                        flag = True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer): 
                                t+=1
                            else:    
                                t44.append(ref1)
                        else: # factual
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f44.append(ref1)
        if n-pre!=1:
            print(ref1)
            print()
print(num[-1]-t,(num[-1]-t)/num[-1])
print(f,f/num[-1])
print('matched num=',n)

 https://www.thegwpf.com/european-climate-declaration/ The European Climate Declaration (ECD) is a document signed by over 500 scientists and professionals in climate and related fields, stating that there is no climate emergency and that the current climate policies are not based on sound science. The declaration emphasizes that the Earth's climate has been changing for centuries and that the current warming period is not unusual when considering the geological history of the Earth. It also states that the impact of human activities on the climate is relatively small compared to natural factors and that the current policies aimed at reducing CO2 emissions will have little effect on the climate. The signatories of the ECD include climatologists, physicists, chemists, biologists, geologists, and other experts from Europe and around the world. They come from universities, research institutions, and private companies, and represent a wide range of scientific disciplines.

 https://www.the

In [604]:
t=0
f=0
t5=[]
f5=[]
n=0
with open(filePath[1]+"dataset_0.5.jsonl") as file1:
    for line1 in file1:
        label=line1.split("label: ")[-1]
        ref1=line1.split("Response: ")[0].split("Reference: ")[1]
        ref1=ref1[:-8]
        pre=n
        flag=False
        with open(filePath[1]+"detections_5.jsonl") as file:
            for line in file:
                answer=line.split("Result: ")[1]
                ref=line.split("P2: ")[0].split("P1: ")[1]
                ref=ref.encode().decode('unicode_escape')
                ref=ref[1:-15]
                if ref.strip()==ref1.strip():
                    flag=True
                    n+=1
                    if 'true' in label: #hallucination
                        if identifyConflict(answer):
                            t+=1
                        else:    
                            t5.append(ref1)
                    else: # factual
                        if identifyConflict(answer):
                            f+=1
                        else:    
                            f5.append(ref1)
#                 if flag==False:
#                     # 
#                     c1=ref.strip().split(".")[0]
#                     c2=ref1.strip().split(".")[0]
#                     if c1.strip()==c2.strip():
#                         flag=True
#                         n+=1
#                         if 'true' in label: #hallucination
#                             if identifyConflict(answer):
#                                 t+=1
#                             else:    
#                                 t5.append(ref1)
#                         else: # factual
#                             if identifyConflict(answer):
#                                 f+=1
#                             else:    
#                                 f5.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split(".")
                    c2=ref1.strip().split(".")
                    if len(c1)>2 and len(c2)>2 and c1[-2]==c2[-2]:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t5.append(ref1)
                        else: # factual
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f5.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\n")[-1]
                    c2=ref1.strip().split("\\\\n")[-1]
                    if c1==c2:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t5.append(ref1)
                        else: # factual
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f5.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                    c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                    if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t5.append(ref1)
                        else: # factual
                            x+=1
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f5.append(ref1)
                if flag==False:
                    uni=ref.strip().split('\\u')[-1][4:]
                    length=len(uni)
                    if ref1.strip()[-length:] == uni:
                        flag = True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer): 
                                t+=1
                            else:    
                                t5.append(ref1)
                        else: # factual
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f5.append(ref1)
        if n-pre!=1:
            print(ref1)
            print()
print(num[-1]-t,(num[-1]-t)/num[-1])
print(f,f/num[-1])
print('matched num=',n)

 https://www.thegwpf.com/european-climate-declaration/ The European Climate Declaration (ECD) is a document signed by over 500 scientists and professionals in climate and related fields, stating that there is no climate emergency and that the current climate policies are not based on sound science. The declaration emphasizes that the Earth's climate has been changing for centuries and that the current warming period is not unusual when considering the geological history of the Earth. It also states that the impact of human activities on the climate is relatively small compared to natural factors and that the current policies aimed at reducing CO2 emissions will have little effect on the climate. The signatories of the ECD include climatologists, physicists, chemists, biologists, geologists, and other experts from Europe and around the world. They come from universities, research institutions, and private companies, and represent a wide range of scientific disciplines.

 https://www.the

In [605]:
t=0
f=0
t55=[]
f55=[]
n=0
with open(filePath[1]+"dataset_0.5.jsonl") as file1:
    for line1 in file1:
        label=line1.split("label: ")[-1]
        ref1=line1.split("Response: ")[0].split("Reference: ")[1]
        ref1=ref1[:-8]
        pre=n
        flag=False
        with open(filePath[1]+"detections_5_1.jsonl") as file:
            for line in file:
                answer=line.split("Result: ")[1]
                ref=line.split("P2: ")[0].split("P1: ")[1]
                ref=ref.encode().decode('unicode_escape')
                ref=ref[1:-15]
                if ref.strip()==ref1.strip():
                    flag=True
                    n+=1
                    if 'true' in label: #hallucination
                        if identifyConflict(answer):
                            t+=1
                        else:    
                            t55.append(ref1)
                    else: # factual
                        if identifyConflict(answer):
                            f+=1
                        else:    
                            f55.append(ref1)
#                 if flag==False:
#                     # 
#                     c1=ref.strip().split(".")[0]
#                     c2=ref1.strip().split(".")[0]
#                     if c1.strip()==c2.strip():
#                         flag=True
#                         n+=1
#                         if 'true' in label: #hallucination
#                             if identifyConflict(answer):
#                                 t+=1
#                             else:    
#                                 t55.append(ref1)
#                         else: # factual
#                             if identifyConflict(answer):
#                                 f+=1
#                             else:    
#                                 f55.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split(".")
                    c2=ref1.strip().split(".")
                    if len(c1)>2 and len(c2)>2 and c1[-2]==c2[-2]:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t55.append(ref1)
                        else: # factual
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f55.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\n")[-1]
                    c2=ref1.strip().split("\\\\n")[-1]
                    if c1==c2:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t55.append(ref1)
                        else: # factual
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f55.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                    c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                    if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t55.append(ref1)
                        else: # factual
                            x+=1
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f55.append(ref1)
                if flag==False:
                    uni=ref.strip().split('\\u')[-1][4:]
                    length=len(uni)
                    if ref1.strip()[-length:] == uni:
                        flag = True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer): 
                                t+=1
                            else:    
                                t55.append(ref1)
                        else: # factual
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f55.append(ref1)
        if n-pre!=1:
            print(ref1)
            print()
print(num[-1]-t,(num[-1]-t)/num[-1])
print(f,f/num[-1])
print('matched num=',n)

 https://www.thegwpf.com/european-climate-declaration/ The European Climate Declaration (ECD) is a document signed by over 500 scientists and professionals in climate and related fields, stating that there is no climate emergency and that the current climate policies are not based on sound science. The declaration emphasizes that the Earth's climate has been changing for centuries and that the current warming period is not unusual when considering the geological history of the Earth. It also states that the impact of human activities on the climate is relatively small compared to natural factors and that the current policies aimed at reducing CO2 emissions will have little effect on the climate. The signatories of the ECD include climatologists, physicists, chemists, biologists, geologists, and other experts from Europe and around the world. They come from universities, research institutions, and private companies, and represent a wide range of scientific disciplines.

 https://www.the

In [606]:
t=0
f=0
t6=[]
f6=[]
n=0
with open(filePath[1]+"dataset_0.5.jsonl") as file1:
    for line1 in file1:
        label=line1.split("label: ")[-1]
        ref1=line1.split("Response: ")[0].split("Reference: ")[1]
        ref1=ref1[:-8]
        pre=n
        flag=False
        with open(filePath[1]+"detections_6.jsonl") as file:
            for line in file:
                answer=line.split("Result: ")[1]
                ref=line.split("P2: ")[0].split("P1: ")[1]
                ref=ref.encode().decode('unicode_escape')
                ref=ref[1:-15]
                if ref.strip()==ref1.strip():
                    flag=True
                    n+=1
                    if 'true' in label: #hallucination
                        if identifyConflict(answer):
                            t+=1
                        else:    
                            t6.append(ref1)
                    else: # factual
                        if identifyConflict(answer):
                            f+=1
                        else:    
                            f6.append(ref1)
#                 if flag==False:
#                     # 
#                     c1=ref.strip().split(".")[0]
#                     c2=ref1.strip().split(".")[0]
#                     if c1.strip()==c2.strip():
#                         flag=True
#                         n+=1
#                         if 'true' in label: #hallucination
#                             if identifyConflict(answer):
#                                 t+=1
#                             else:    
#                                 t6.append(ref1)
#                         else: # factual
#                             if identifyConflict(answer):
#                                 f+=1
#                             else:    
#                                 f6.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split(".")
                    c2=ref1.strip().split(".")
                    if len(c1)>2 and len(c2)>2 and c1[-2]==c2[-2]:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t6.append(ref1)
                        else: # factual
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f6.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\n")[-1]
                    c2=ref1.strip().split("\\\\n")[-1]
                    if c1==c2:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t6.append(ref1)
                        else: # factual
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f6.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                    c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                    if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t6.append(ref1)
                        else: # factual
                            x+=1
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f6.append(ref1)
                if flag==False:
                    uni=ref.strip().split('\\u')[-1][4:]
                    length=len(uni)
                    if ref1.strip()[-length:] == uni:
                        flag = True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer): 
                                t+=1
                            else:    
                                t6.append(ref1)
                        else: # factual
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f6.append(ref1)
        if n-pre!=1:
            print(ref1)
            print()
print(num[-1]-t,(num[-1]-t)/num[-1])
print(f,f/num[-1])
print('matched num=',n)

 https://www.thegwpf.com/european-climate-declaration/ The European Climate Declaration (ECD) is a document signed by over 500 scientists and professionals in climate and related fields, stating that there is no climate emergency and that the current climate policies are not based on sound science. The declaration emphasizes that the Earth's climate has been changing for centuries and that the current warming period is not unusual when considering the geological history of the Earth. It also states that the impact of human activities on the climate is relatively small compared to natural factors and that the current policies aimed at reducing CO2 emissions will have little effect on the climate. The signatories of the ECD include climatologists, physicists, chemists, biologists, geologists, and other experts from Europe and around the world. They come from universities, research institutions, and private companies, and represent a wide range of scientific disciplines.

 https://www.the

In [607]:
t=0
f=0
t66=[]
f66=[]
n=0
x=0
with open(filePath[1]+"dataset_0.5.jsonl") as file1:
    for line1 in file1:
        label=line1.split("label: ")[-1]
        ref1=line1.split("Response: ")[0].split("Reference: ")[1]
        ref1=ref1[:-8]
        pre=n
        flag=False
        with open(filePath[1]+"detections_6_1.jsonl") as file:
            for line in file:
                answer=line.split("Result: ")[1]
                ref=line.split("P2: ")[0].split("P1: ")[1]
                ref=ref.encode().decode('unicode_escape')
                ref=ref[1:-15]
                if ref.strip()==ref1.strip():
                    flag=True
                    n+=1
                    if 'true' in label: #hallucination
                        if identifyConflict(answer):
                            t+=1
                        else:    
                            t66.append(ref1)
                    else: # factual
                        x+=1
                        if identifyConflict(answer):
                            f+=1
                        else:    
                            f66.append(ref1)
#                 if flag==False:
#                     # 
#                     c1=ref.strip().split(".")[0]
#                     c2=ref1.strip().split(".")[0]
#                     if c1.strip()==c2.strip():
#                         flag=True
#                         n+=1
#                         if 'true' in label: #hallucination
#                             if identifyConflict(answer):
#                                 t+=1
#                             else:    
#                                 t66.append(ref1)
#                         else: # factual
#                             x+=1
#                             if identifyConflict(answer):
#                                 f+=1
#                             else:    
#                                 f66.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split(".")
                    c2=ref1.strip().split(".")
                    if len(c1)>2 and len(c2)>2 and c1[-2]==c2[-2]:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t66.append(ref1)
                        else: # factual
                            x+=1
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f66.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\n")[-1]
                    c2=ref1.strip().split("\\\\n")[-1]
                    if c1==c2:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t66.append(ref1)
                        else: # factual
                            x+=1
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f66.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                    c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                    if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t66.append(ref1)
                        else: # factual
                            x+=1
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f66.append(ref1)
                if flag==False:
                    uni=ref.strip().split('\\u')[-1][4:]
                    length=len(uni)
                    if ref1.strip()[-length:] == uni:
                        flag = True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer): 
                                t+=1
                            else:    
                                t66.append(ref1)
                        else: # factual
                            x+=1
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f66.append(ref1)
        if n-pre!=1:
            print(ref1)
            print()
print(num[-1]-t,(num[-1]-t)/num[-1])
print(f,f/num[-1])
print('matched num=',n)
print(x)

 https://www.thegwpf.com/european-climate-declaration/ The European Climate Declaration (ECD) is a document signed by over 500 scientists and professionals in climate and related fields, stating that there is no climate emergency and that the current climate policies are not based on sound science. The declaration emphasizes that the Earth's climate has been changing for centuries and that the current warming period is not unusual when considering the geological history of the Earth. It also states that the impact of human activities on the climate is relatively small compared to natural factors and that the current policies aimed at reducing CO2 emissions will have little effect on the climate. The signatories of the ECD include climatologists, physicists, chemists, biologists, geologists, and other experts from Europe and around the world. They come from universities, research institutions, and private companies, and represent a wide range of scientific disciplines.

 https://www.the

In [608]:
t=0
f=0
t7=[]
f7=[]
n=0
x=0
with open(filePath[1]+"dataset_0.5.jsonl") as file1:
    for line1 in file1:
        label=line1.split("label: ")[-1]
        ref1=line1.split("Response: ")[0].split("Reference: ")[1]
        ref1=ref1[:-8]
        pre=n
        flag=False
        with open(filePath[1]+"detections_7.jsonl") as file:
            for line in file:
                answer=line.split("Result: ")[1]
                ref=line.split("P2: ")[0].split("P1: ")[1]
                ref=ref.encode().decode('unicode_escape')
                ref=ref[1:-15]
                if ref.strip()==ref1.strip():
                    flag=True
                    n+=1
                    if 'true' in label: #hallucination
                        if identifyConflict(answer):
                            t+=1
                        else:    
                            t7.append(ref1)
                    else: # factual
                        x+=1
                        if identifyConflict(answer):
                            f+=1
                        else:    
                            f7.append(ref1)
#                 if flag==False:
#                     # 
#                     c1=ref.strip().split(".")[0]
#                     c2=ref1.strip().split(".")[0]
#                     if c1.strip()==c2.strip():
#                         flag=True
#                         n+=1
#                         if 'true' in label: #hallucination
#                             if identifyConflict(answer):
#                                 t+=1
#                             else:    
#                                 t7.append(ref1)
#                         else: # factual
#                             x+=1
#                             if identifyConflict(answer):
#                                 f+=1
#                             else:    
#                                 f7.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split(".")
                    c2=ref1.strip().split(".")
                    if len(c1)>2 and len(c2)>2 and c1[-2]==c2[-2]:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t7.append(ref1)
                        else: # factual
                            x+=1
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f7.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\n")[-1]
                    c2=ref1.strip().split("\\\\n")[-1]
                    if c1==c2:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t7.append(ref1)
                        else: # factual
                            x+=1
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f7.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                    c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                    if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                        flag=True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t7.append(ref1)
                        else: # factual
                            x+=1
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f7.append(ref1)
                if flag==False:
                    uni=ref.strip().split('\\u')[-1][4:]
                    length=len(uni)
                    if ref1.strip()[-length:] == uni:
                        flag = True
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer): 
                                t+=1
                            else:    
                                t7.append(ref1)
                        else: # factual
                            x+=1
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f7.append(ref1)
        if n-pre!=1:
            print(n-pre)
            print(ref1)
            print()
print(num[-1]-t,(num[-1]-t)/num[-1])
print(f,f/num[-1])
print('matched num=',n)
print(x)

2
 https://www.thegwpf.com/european-climate-declaration/ The European Climate Declaration (ECD) is a document signed by over 500 scientists and professionals in climate and related fields, stating that there is no climate emergency and that the current climate policies are not based on sound science. The declaration emphasizes that the Earth's climate has been changing for centuries and that the current warming period is not unusual when considering the geological history of the Earth. It also states that the impact of human activities on the climate is relatively small compared to natural factors and that the current policies aimed at reducing CO2 emissions will have little effect on the climate. The signatories of the ECD include climatologists, physicists, chemists, biologists, geologists, and other experts from Europe and around the world. They come from universities, research institutions, and private companies, and represent a wide range of scientific disciplines.

2
 https://www

In [625]:
tlist=[t1,t2,t3,t4,t5,t6,t7,t11,t22,t33,t44,t55,t66]
flist=[f1,f2,f3,f4,f5,f6,f7,f11,f22,f33,f44,f55,f66]
#      0   1  2  3  4  5  6  7   8   9   10  11  12
minV=0
for kk in range(13-11):
    for jj in range(kk+1,13-10):
        for ii in range(jj+1,13-9):
            for i in range(ii+1,13-8):
                for j in range(i+1,13-7):
                     for k in range(j+1,13-6):
                        for m in range(k+1,13-5):
                            for n in range(m+1,13-4):
                                for q in range(n+1,13-3):
                                    for w in range(q+1,13-2):
                                        for z in range(w+1,13-1):
                                            for y in range(z+1,13-0):
                                                a=tlist[kk]+tlist[jj]+tlist[ii]+tlist[i]+tlist[j]+tlist[k]+tlist[m]+tlist[n]+tlist[q]+tlist[w]+tlist[z]+tlist[y]#+tlist[p] 
                                                a=list(set(a))
                                                aa=len(a)
                                                a=flist[kk]+flist[jj]+flist[ii]+flist[i]+flist[j]+flist[k]+flist[m]+flist[n]+flist[q]+flist[w]+flist[z]+flist[y]#+flist[p] 
                                                a=list(set(a))
                                                bb=len(a)
                                                TN=(num[-1]-bb)/(num[-1]*2)
                                                TP=aa/(num[-1]*2)
                                                FN=(num[-1]-aa)/(num[-1]*2)
                                                FP=bb/(num[-1]*2)
                                                F1=2*TP/(2*TP+FP+FN)
                                                if TN+TP>=minV:
                                                    print(kk,jj,ii,i,j,k,m,n,q,w,z)#,y,p)
                                                    print(TN+TP)
                                                    print(F1)
                                                    minV=TN+TP

0 1 2 3 4 5 6 7 8 9 10
0.5
0.6226012793176973
0 1 2 3 4 5 6 7 8 10 11
0.5028248587570621
0.6271186440677965
0 1 2 3 4 6 7 8 9 10 11
0.5056497175141242
0.6284501061571125
0 1 2 3 5 6 7 8 9 10 11
0.5056497175141244
0.6268656716417911


In [610]:
a=t1+t2+t3+t4+t5+t6+t7+t11+t22+t33+t44+t55+t66 # 
res=set(a)
a=list(res)
print(len(a))
aa=len(a)
print(len(a)/num[-1])

149
0.8418079096045198


In [611]:
a=f1+f2+f3+f4+f5+f6+f7+f11+f22+f33+f44+f55+f66 # 
res=set(a)
a=list(res)
print(len(a))
bb=len(a)
print((num[-1]-len(a))/num[-1])

150
0.15254237288135594


In [612]:
nu=num[-1]
all_nu=nu*2
# FN:将正类错误预测为负类数，即未检测出来的幻觉数据比例
FN=(nu-aa)/all_nu
print('FN',FN)
# FP:将负类错误预测为正类数,即误把factual数据判定为有幻觉的比例
FP=bb/all_nu
print('FP',FP)
# TN:将负类正确预测为负类数,即检测出的factual数据比例
TN=(nu-bb)/all_nu
print('TN',TN)
# TP:将正类正确预测为正类数,即检测出来的幻觉数据比例
TP=aa/all_nu
print('TP',TP)

FN 0.07909604519774012
FP 0.423728813559322
TN 0.07627118644067797
TP 0.4209039548022599


In [613]:
Accuracy=TN+TP
print(Accuracy)
Precision=TP/(TP+FP)
print(Precision)
Recall=TP/(TP+FN)
print(Recall)
F1=2*TP/(2*TP+FP+FN)
print(F1)

0.4971751412429379
0.4983277591973244
0.8418079096045198
0.6260504201680673


# climate+0.9

In [514]:
t=0
f=0
t1=[]
f1=[]
n=0
x=0
with open(filePath[0]+"dataset_0.9.jsonl") as file1:
    for line1 in file1:
        label=line1.split("label: ")[-1]
        ref1=line1.split("Response: ")[0].split("Reference: ")
        if len(ref1)==2:
            ref1=ref1[1]
        else:
            ref1=ref1[1]+"Reference: "+ref1[2]
        pre=n
        flag=False
        ref1=ref1[:-8]
        with open("detections_1_3.jsonl") as file:
            mark=[]
            for line in file:
                answer=line.split("Result: ")[1]
                ref=line.split("P2: ")[0].split("P1: ")[1]
                ref=ref.encode().decode('unicode_escape')
                ref=ref[1:-15]
                if "The Real Inconvenient Truth" in ref:
                    ref+="ng"
                if ref.strip()==ref1.strip():
                    flag=True
                    n+=1
                    if 'true' in label: #hallucination
                        x+=1
                        if identifyConflict(answer):
                            t+=1
                        else:    
                            t1.append(ref1)
                    else: # factual
                        if identifyConflict(answer): 
                            f+=1
                        else:    
                            f1.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split(",")[0]
                    c2=ref1.strip().split(",")[0]
                    if c1.strip()==c2.strip():
                        flag=True
                        n+=1
                        mark.append(',')
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer): 
                                t+=1
                            else:    
                                t1.append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                f1.append(ref1)
#                 if flag==False:
#                     # 
#                     c1=ref.strip().split(".")
#                     c2=ref1.strip().split(".")
#                     if len(c1)>2 and len(c2)>2 and c1[-2]==c2[-2]:
#                         flag=True
#                         n+=1
#                         mark.append('.')
#                         if 'true' in label: #hallucination
#                             x+=1
#                             if identifyConflict(answer): 
#                                 t+=1
#                             else:    
#                                 t1.append(ref1)
#                         else: # factual
#                             if identifyConflict(answer): 
#                                 f+=1
#                             else:    
#                                 f1.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\n")
                    c2=ref1.strip().split("\\\\n")
                    if len(c1)>2 and len(c2)>2 and c1[-1]==c2[-1]:
                        flag=True
                        mark.append('n')
                        n+=1
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer): 
                                t+=1
                            else:    
                                t1.append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                f1.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                    c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                    if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                        flag=True
                        mark.append('\"')
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t1.append(ref1)
                        else: # factual
                            x+=1
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f1.append(ref1)
                if flag==False:
                    uni=ref.strip().split('\\\\\\\\u')
                    if len(uni)>1 and "d Unreliability and Electricity Prices: Th" not in ref:
                        uni=uni[-2][4:]
                        length=len(uni)
                        #if ref1.strip()[-length:] == uni:
                        if uni in ref1.strip():
                            flag = True
                            mark.append('u')
                            mark.append(ref)
                            n+=1
                            if 'true' in label: #hallucination
                                x+=1
                                if identifyConflict(answer): 
                                    t+=1
                                else:    
                                    t1.append(ref1)
                            else: # factual
                                if identifyConflict(answer): 
                                    f+=1
                                else:    
                                    f1.append(ref1)
        if n-pre!=1:
            print(mark)
            print(n-pre)
            print(ref1)
            print()
print(x)
print(len(t1))
print(t)
print(num[-1]-t,(num[-1]-t)/num[-1])
print(f,f/num[-1])
print('matched num=',n)

[]
2
 https://www.thegwpf.com/european-climate-declaration/ The European Climate Declaration (ECD) is a public statement on climate change signed by over 500 scientists and professionals in climate and related fields from Europe and beyond. The declaration asserts that a human-caused climate emergency does not exist and emphasizes the need for a more balanced and nuanced approach to climate policy. It highlights the benefits of carbon dioxide for plant growth and the lack of evidence supporting extreme weather events being caused by human activity. The ECD acknowledges the potential for climate variability but stresses that natural factors such as solar activity and changes in global and regional temperature patterns are also important drivers of climate change. In conclusion, this reference supports a dissenting view on climate change and its causes, challenging the widely accepted position that human activities, particularly burning fossil fuels, are primarily responsible for global 

In [515]:
t=0
f=0
t11=[]
f11=[]
n=0
x=0
with open(filePath[0]+"dataset_0.9.jsonl") as file1:
    for line1 in file1:
        label=line1.split("label: ")[-1]
        ref1=line1.split("Response: ")[0].split("Reference: ")
        if len(ref1)==2:
            ref1=ref1[1]
        else:
            ref1=ref1[1]+"Reference: "+ref1[2]
        pre=n
        flag=False
        ref1=ref1[:-8]
        with open(filePath[0]+"detections_1_1.jsonl") as file:
            mark=[]
            for line in file:
                answer=line.split("Result: ")[1]
                ref=line.split("P2: ")[0].split("P1: ")[1]
                ref=ref.encode().decode('unicode_escape')
                ref=ref[1:-15]
                if "The Real Inconvenient Truth" in ref:
                    ref+="ng"
                if ref.strip()==ref1.strip():
                    flag=True
                    n+=1
                    if 'true' in label: #hallucination
                        x+=1
                        if identifyConflict(answer):
                            t+=1
                        else:    
                            t11.append(ref1)
                    else: # factual
                        if identifyConflict(answer): 
                            f+=1
                        else:    
                            f11.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split(",")[0]
                    c2=ref1.strip().split(",")[0]
                    if c1.strip()==c2.strip():
                        flag=True
                        n+=1
                        mark.append(',')
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer): 
                                t+=1
                            else:    
                                t11.append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                f11.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\n")
                    c2=ref1.strip().split("\\\\n")
                    if len(c1)>2 and len(c2)>2 and c1[-1]==c2[-1]:
                        flag=True
                        mark.append('n')
                        n+=1
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer): 
                                t+=1
                            else:    
                                t11.append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                f11.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                    c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                    if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                        flag=True
                        mark.append('\"')
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t11.append(ref1)
                        else: # factual
                            x+=1
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f11.append(ref1)
                if flag==False:
                    uni=ref.strip().split('\\\\\\\\u')
                    if len(uni)>1 and "d Unreliability and Electricity Prices: Th" not in ref:
                        uni=uni[-2][4:]
                        length=len(uni)
                        #if ref1.strip()[-length:] == uni:
                        if uni in ref1.strip():
                            flag = True
                            mark.append('u')
                            mark.append(ref)
                            n+=1
                            if 'true' in label: #hallucination
                                x+=1
                                if identifyConflict(answer): 
                                    t+=1
                                else:    
                                    t11.append(ref1)
                            else: # factual
                                if identifyConflict(answer): 
                                    f+=1
                                else:    
                                    f11.append(ref1)
        if n-pre!=1:
            print(mark)
            print(n-pre)
            print(ref1)
            print()
print(x)
print(len(t1))
print(t)
print(num[-1]-t,(num[-1]-t)/num[-1])
print(f,f/num[-1])
print('matched num=',n)

[]
2
 https://www.thegwpf.com/european-climate-declaration/ The European Climate Declaration (ECD) is a public statement on climate change signed by over 500 scientists and professionals in climate and related fields from Europe and beyond. The declaration asserts that a human-caused climate emergency does not exist and emphasizes the need for a more balanced and nuanced approach to climate policy. It highlights the benefits of carbon dioxide for plant growth and the lack of evidence supporting extreme weather events being caused by human activity. The ECD acknowledges the potential for climate variability but stresses that natural factors such as solar activity and changes in global and regional temperature patterns are also important drivers of climate change. In conclusion, this reference supports a dissenting view on climate change and its causes, challenging the widely accepted position that human activities, particularly burning fossil fuels, are primarily responsible for global 

In [516]:
t=0
f=0
t2=[]
f2=[]
n=0
x=0
with open(filePath[0]+"dataset_0.9.jsonl") as file1:
    for line1 in file1:
        label=line1.split("label: ")[-1]
        ref1=line1.split("Response: ")[0].split("Reference: ")
        if len(ref1)==2:
            ref1=ref1[1]
        else:
            ref1=ref1[1]+"Reference: "+ref1[2]
        pre=n
        flag=False
        ref1=ref1[:-8]
        with open("detections_2_3.jsonl") as file:
            mark=[]
            for line in file:
                answer=line.split("Result: ")[1]
                ref=line.split("P2: ")[0].split("P1: ")[1]
                ref=ref.encode().decode('unicode_escape')
                ref=ref[1:-15]
                if "The Real Inconvenient Truth" in ref:
                    ref+="ng"
                if ref.strip()==ref1.strip():
                    flag=True
                    n+=1
                    if 'true' in label: #hallucination
                        x+=1
                        if identifyConflict(answer):
                            t+=1
                        else:    
                            t2.append(ref1)
                    else: # factual
                        if identifyConflict(answer): 
                            f+=1
                        else:    
                            f2.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split(",")[0]
                    c2=ref1.strip().split(",")[0]
                    if c1.strip()==c2.strip():
                        flag=True
                        n+=1
                        mark.append(',')
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer): 
                                t+=1
                            else:    
                                t2.append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                f2.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\n")
                    c2=ref1.strip().split("\\\\n")
                    if len(c1)>2 and len(c2)>2 and c1[-1]==c2[-1]:
                        flag=True
                        mark.append('n')
                        n+=1
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer): 
                                t+=1
                            else:    
                                t2.append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                f2.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                    c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                    if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                        flag=True
                        mark.append('\"')
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t2.append(ref1)
                        else: # factual
                            x+=1
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f2.append(ref1)
                if flag==False:
                    uni=ref.strip().split('\\\\\\\\u')
                    if len(uni)>1 and "d Unreliability and Electricity Prices: Th" not in ref:
                        uni=uni[-2][4:]
                        length=len(uni)
                        #if ref1.strip()[-length:] == uni:
                        if uni in ref1.strip():
                            flag = True
                            mark.append('u')
                            mark.append(ref)
                            n+=1
                            if 'true' in label: #hallucination
                                x+=1
                                if identifyConflict(answer): 
                                    t+=1
                                else:    
                                    t2.append(ref1)
                            else: # factual
                                if identifyConflict(answer): 
                                    f+=1
                                else:    
                                    f2.append(ref1)
        if n-pre!=1:
            print(mark)
            print(n-pre)
            print(ref1)
            print()
print(x)
print(len(t1))
print(t)
print(num[-1]-t,(num[-1]-t)/num[-1])
print(f,f/num[-1])
print('matched num=',n)

[]
2
 https://www.thegwpf.com/european-climate-declaration/ The European Climate Declaration (ECD) is a public statement on climate change signed by over 500 scientists and professionals in climate and related fields from Europe and beyond. The declaration asserts that a human-caused climate emergency does not exist and emphasizes the need for a more balanced and nuanced approach to climate policy. It highlights the benefits of carbon dioxide for plant growth and the lack of evidence supporting extreme weather events being caused by human activity. The ECD acknowledges the potential for climate variability but stresses that natural factors such as solar activity and changes in global and regional temperature patterns are also important drivers of climate change. In conclusion, this reference supports a dissenting view on climate change and its causes, challenging the widely accepted position that human activities, particularly burning fossil fuels, are primarily responsible for global 

In [517]:
t=0
f=0
t22=[]
f22=[]
n=0
x=0
with open(filePath[0]+"dataset_0.9.jsonl") as file1:
    for line1 in file1:
        label=line1.split("label: ")[-1]
        ref1=line1.split("Response: ")[0].split("Reference: ")
        if len(ref1)==2:
            ref1=ref1[1]
        else:
            ref1=ref1[1]+"Reference: "+ref1[2]
        pre=n
        flag=False
        ref1=ref1[:-8]
        with open(filePath[0]+"detections_2_1.jsonl") as file:
            mark=[]
            for line in file:
                answer=line.split("Result: ")[1]
                ref=line.split("P2: ")[0].split("P1: ")[1]
                ref=ref.encode().decode('unicode_escape')
                ref=ref[1:-15]
                if "The Real Inconvenient Truth" in ref:
                    ref+="ng"
                if ref.strip()==ref1.strip():
                    flag=True
                    n+=1
                    if 'true' in label: #hallucination
                        x+=1
                        if identifyConflict(answer):
                            t+=1
                        else:    
                            t22.append(ref1)
                    else: # factual
                        if identifyConflict(answer): 
                            f+=1
                        else:    
                            f22.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split(",")[0]
                    c2=ref1.strip().split(",")[0]
                    if c1.strip()==c2.strip():
                        flag=True
                        n+=1
                        mark.append(',')
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer): 
                                t+=1
                            else:    
                                t22.append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                f22.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\n")
                    c2=ref1.strip().split("\\\\n")
                    if len(c1)>2 and len(c2)>2 and c1[-1]==c2[-1]:
                        flag=True
                        mark.append('n')
                        n+=1
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer): 
                                t+=1
                            else:    
                                t22.append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                f22.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                    c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                    if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                        flag=True
                        mark.append('\"')
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t22.append(ref1)
                        else: # factual
                            x+=1
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f22.append(ref1)
                if flag==False:
                    uni=ref.strip().split('\\\\\\\\u')
                    if len(uni)>1 and "d Unreliability and Electricity Prices: Th" not in ref:
                        uni=uni[-2][4:]
                        length=len(uni)
                        #if ref1.strip()[-length:] == uni:
                        if uni in ref1.strip():
                            flag = True
                            mark.append('u')
                            mark.append(ref)
                            n+=1
                            if 'true' in label: #hallucination
                                x+=1
                                if identifyConflict(answer): 
                                    t+=1
                                else:    
                                    t22.append(ref1)
                            else: # factual
                                if identifyConflict(answer): 
                                    f+=1
                                else:    
                                    f22.append(ref1)
        if n-pre!=1:
            print(mark)
            print(n-pre)
            print(ref1)
            print()
print(x)
print(len(t1))
print(t)
print(num[-1]-t,(num[-1]-t)/num[-1])
print(f,f/num[-1])
print('matched num=',n)

[]
2
 https://www.thegwpf.com/european-climate-declaration/ The European Climate Declaration (ECD) is a public statement on climate change signed by over 500 scientists and professionals in climate and related fields from Europe and beyond. The declaration asserts that a human-caused climate emergency does not exist and emphasizes the need for a more balanced and nuanced approach to climate policy. It highlights the benefits of carbon dioxide for plant growth and the lack of evidence supporting extreme weather events being caused by human activity. The ECD acknowledges the potential for climate variability but stresses that natural factors such as solar activity and changes in global and regional temperature patterns are also important drivers of climate change. In conclusion, this reference supports a dissenting view on climate change and its causes, challenging the widely accepted position that human activities, particularly burning fossil fuels, are primarily responsible for global 

In [518]:
t=0
f=0
t3=[]
f3=[]
n=0
x=0
with open(filePath[0]+"dataset_0.9.jsonl") as file1:
    for line1 in file1:
        label=line1.split("label: ")[-1]
        ref1=line1.split("Response: ")[0].split("Reference: ")
        if len(ref1)==2:
            ref1=ref1[1]
        else:
            ref1=ref1[1]+"Reference: "+ref1[2]
        pre=n
        flag=False
        ref1=ref1[:-8]
        with open("detections_3_3.jsonl") as file:
            mark=[]
            for line in file:
                answer=line.split("Result: ")[1]
                ref=line.split("P2: ")[0].split("P1: ")[1]
                ref=ref.encode().decode('unicode_escape')
                ref=ref[1:-15]
                if "The Real Inconvenient Truth" in ref:
                    ref+="ng"
                if ref.strip()==ref1.strip():
                    flag=True
                    n+=1
                    if 'true' in label: #hallucination
                        x+=1
                        if identifyConflict(answer):
                            t+=1
                        else:    
                            t3.append(ref1)
                    else: # factual
                        if identifyConflict(answer): 
                            f+=1
                        else:    
                            f3.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split(",")[0]
                    c2=ref1.strip().split(",")[0]
                    if c1.strip()==c2.strip():
                        flag=True
                        n+=1
                        mark.append(',')
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer): 
                                t+=1
                            else:    
                                t3.append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                f3.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\n")
                    c2=ref1.strip().split("\\\\n")
                    if len(c1)>2 and len(c2)>2 and c1[-1]==c2[-1]:
                        flag=True
                        mark.append('n')
                        n+=1
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer): 
                                t+=1
                            else:    
                                t3.append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                f3.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                    c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                    if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                        flag=True
                        mark.append('\"')
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t3.append(ref1)
                        else: # factual
                            x+=1
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f3.append(ref1)
                if flag==False:
                    uni=ref.strip().split('\\\\\\\\u')
                    if len(uni)>1 and "d Unreliability and Electricity Prices: Th" not in ref:
                        uni=uni[-2][4:]
                        length=len(uni)
                        #if ref1.strip()[-length:] == uni:
                        if uni in ref1.strip():
                            flag = True
                            mark.append('u')
                            mark.append(ref)
                            n+=1
                            if 'true' in label: #hallucination
                                x+=1
                                if identifyConflict(answer): 
                                    t+=1
                                else:    
                                    t3.append(ref1)
                            else: # factual
                                if identifyConflict(answer): 
                                    f+=1
                                else:    
                                    f3.append(ref1)
        if n-pre!=1:
            print(mark)
            print(n-pre)
            print(ref1)
            print()
print(x)
print(len(t1))
print(t)
print(num[-1]-t,(num[-1]-t)/num[-1])
print(f,f/num[-1])
print('matched num=',n)

[]
2
 https://www.thegwpf.com/european-climate-declaration/ The European Climate Declaration (ECD) is a public statement on climate change signed by over 500 scientists and professionals in climate and related fields from Europe and beyond. The declaration asserts that a human-caused climate emergency does not exist and emphasizes the need for a more balanced and nuanced approach to climate policy. It highlights the benefits of carbon dioxide for plant growth and the lack of evidence supporting extreme weather events being caused by human activity. The ECD acknowledges the potential for climate variability but stresses that natural factors such as solar activity and changes in global and regional temperature patterns are also important drivers of climate change. In conclusion, this reference supports a dissenting view on climate change and its causes, challenging the widely accepted position that human activities, particularly burning fossil fuels, are primarily responsible for global 

In [519]:
t=0
f=0
t33=[]
f33=[]
n=0
x=0
with open(filePath[0]+"dataset_0.9.jsonl") as file1:
    for line1 in file1:
        label=line1.split("label: ")[-1]
        ref1=line1.split("Response: ")[0].split("Reference: ")
        if len(ref1)==2:
            ref1=ref1[1]
        else:
            ref1=ref1[1]+"Reference: "+ref1[2]
        pre=n
        flag=False
        ref1=ref1[:-8]
        with open(filePath[0]+"detections_3_1.jsonl") as file:
            mark=[]
            for line in file:
                answer=line.split("Result: ")[1]
                ref=line.split("P2: ")[0].split("P1: ")[1]
                ref=ref.encode().decode('unicode_escape')
                ref=ref[1:-15]
                if "The Real Inconvenient Truth" in ref:
                    ref+="ng"
                if ref.strip()==ref1.strip():
                    flag=True
                    n+=1
                    if 'true' in label: #hallucination
                        x+=1
                        if identifyConflict(answer):
                            t+=1
                        else:    
                            t33.append(ref1)
                    else: # factual
                        if identifyConflict(answer): 
                            f+=1
                        else:    
                            f33.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split(",")[0]
                    c2=ref1.strip().split(",")[0]
                    if c1.strip()==c2.strip():
                        flag=True
                        n+=1
                        mark.append(',')
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer): 
                                t+=1
                            else:    
                                t33.append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                f33.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\n")
                    c2=ref1.strip().split("\\\\n")
                    if len(c1)>2 and len(c2)>2 and c1[-1]==c2[-1]:
                        flag=True
                        mark.append('n')
                        n+=1
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer): 
                                t+=1
                            else:    
                                t33.append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                f33.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                    c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                    if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                        flag=True
                        mark.append('\"')
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t33.append(ref1)
                        else: # factual
                            x+=1
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f33.append(ref1)
                if flag==False:
                    uni=ref.strip().split('\\\\\\\\u')
                    if len(uni)>1 and "d Unreliability and Electricity Prices: Th" not in ref:
                        uni=uni[-2][4:]
                        length=len(uni)
                        #if ref1.strip()[-length:] == uni:
                        if uni in ref1.strip():
                            flag = True
                            mark.append('u')
                            mark.append(ref)
                            n+=1
                            if 'true' in label: #hallucination
                                x+=1
                                if identifyConflict(answer): 
                                    t+=1
                                else:    
                                    t33.append(ref1)
                            else: # factual
                                if identifyConflict(answer): 
                                    f+=1
                                else:    
                                    f33.append(ref1)
        if n-pre!=1:
            print(mark)
            print(n-pre)
            print(ref1)
            print()
print(x)
print(len(t1))
print(t)
print(num[-1]-t,(num[-1]-t)/num[-1])
print(f,f/num[-1])
print('matched num=',n)

[]
2
 https://www.thegwpf.com/european-climate-declaration/ The European Climate Declaration (ECD) is a public statement on climate change signed by over 500 scientists and professionals in climate and related fields from Europe and beyond. The declaration asserts that a human-caused climate emergency does not exist and emphasizes the need for a more balanced and nuanced approach to climate policy. It highlights the benefits of carbon dioxide for plant growth and the lack of evidence supporting extreme weather events being caused by human activity. The ECD acknowledges the potential for climate variability but stresses that natural factors such as solar activity and changes in global and regional temperature patterns are also important drivers of climate change. In conclusion, this reference supports a dissenting view on climate change and its causes, challenging the widely accepted position that human activities, particularly burning fossil fuels, are primarily responsible for global 

In [520]:
t=0
f=0
t4=[]
f4=[]
n=0
x=0
with open(filePath[0]+"dataset_0.9.jsonl") as file1:
    for line1 in file1:
        label=line1.split("label: ")[-1]
        ref1=line1.split("Response: ")[0].split("Reference: ")
        if len(ref1)==2:
            ref1=ref1[1]
        else:
            ref1=ref1[1]+"Reference: "+ref1[2]
        pre=n
        flag=False
        ref1=ref1[:-8]
        with open("detections_4_3.jsonl") as file:
            mark=[]
            for line in file:
                answer=line.split("Result: ")[1]
                ref=line.split("P2: ")[0].split("P1: ")[1]
                ref=ref.encode().decode('unicode_escape')
                ref=ref[1:-15]
                if "The Real Inconvenient Truth" in ref:
                    ref+="ng"
                if ref.strip()==ref1.strip():
                    flag=True
                    n+=1
                    if 'true' in label: #hallucination
                        x+=1
                        if identifyConflict(answer):
                            t+=1
                        else:    
                            t4.append(ref1)
                    else: # factual
                        if identifyConflict(answer): 
                            f+=1
                        else:    
                            f4.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split(",")[0]
                    c2=ref1.strip().split(",")[0]
                    if c1.strip()==c2.strip():
                        flag=True
                        n+=1
                        mark.append(',')
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer): 
                                t+=1
                            else:    
                                t4.append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                f4.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\n")
                    c2=ref1.strip().split("\\\\n")
                    if len(c1)>2 and len(c2)>2 and c1[-1]==c2[-1]:
                        flag=True
                        mark.append('n')
                        n+=1
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer): 
                                t+=1
                            else:    
                                t4.append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                f4.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                    c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                    if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                        flag=True
                        mark.append('\"')
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t4.append(ref1)
                        else: # factual
                            x+=1
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f4.append(ref1)
                if flag==False:
                    uni=ref.strip().split('\\\\\\\\u')
                    if len(uni)>1 and "d Unreliability and Electricity Prices: Th" not in ref:
                        uni=uni[-2][4:]
                        length=len(uni)
                        #if ref1.strip()[-length:] == uni:
                        if uni in ref1.strip():
                            flag = True
                            mark.append('u')
                            mark.append(ref)
                            n+=1
                            if 'true' in label: #hallucination
                                x+=1
                                if identifyConflict(answer): 
                                    t+=1
                                else:    
                                    t4.append(ref1)
                            else: # factual
                                if identifyConflict(answer): 
                                    f+=1
                                else:    
                                    f4.append(ref1)
        if n-pre!=1:
            print(mark)
            print(n-pre)
            print(ref1)
            print()
print(x)
print(len(t1))
print(t)
print(num[-1]-t,(num[-1]-t)/num[-1])
print(f,f/num[-1])
print('matched num=',n)

[]
2
 https://www.thegwpf.com/european-climate-declaration/ The European Climate Declaration (ECD) is a public statement on climate change signed by over 500 scientists and professionals in climate and related fields from Europe and beyond. The declaration asserts that a human-caused climate emergency does not exist and emphasizes the need for a more balanced and nuanced approach to climate policy. It highlights the benefits of carbon dioxide for plant growth and the lack of evidence supporting extreme weather events being caused by human activity. The ECD acknowledges the potential for climate variability but stresses that natural factors such as solar activity and changes in global and regional temperature patterns are also important drivers of climate change. In conclusion, this reference supports a dissenting view on climate change and its causes, challenging the widely accepted position that human activities, particularly burning fossil fuels, are primarily responsible for global 

In [521]:
t=0
f=0
t44=[]
f44=[]
n=0
x=0
with open(filePath[0]+"dataset_0.9.jsonl") as file1:
    for line1 in file1:
        label=line1.split("label: ")[-1]
        ref1=line1.split("Response: ")[0].split("Reference: ")
        if len(ref1)==2:
            ref1=ref1[1]
        else:
            ref1=ref1[1]+"Reference: "+ref1[2]
        pre=n
        flag=False
        ref1=ref1[:-8]
        with open(filePath[0]+"detections_4_1.jsonl") as file:
            mark=[]
            for line in file:
                answer=line.split("Result: ")[1]
                ref=line.split("P2: ")[0].split("P1: ")[1]
                ref=ref.encode().decode('unicode_escape')
                ref=ref[1:-15]
                if "The Real Inconvenient Truth" in ref:
                    ref+="ng"
                if ref.strip()==ref1.strip():
                    flag=True
                    n+=1
                    if 'true' in label: #hallucination
                        x+=1
                        if identifyConflict(answer):
                            t+=1
                        else:    
                            t44.append(ref1)
                    else: # factual
                        if identifyConflict(answer): 
                            f+=1
                        else:    
                            f44.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split(",")[0]
                    c2=ref1.strip().split(",")[0]
                    if c1.strip()==c2.strip():
                        flag=True
                        n+=1
                        mark.append(',')
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer): 
                                t+=1
                            else:    
                                t44.append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                f44.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\n")
                    c2=ref1.strip().split("\\\\n")
                    if len(c1)>2 and len(c2)>2 and c1[-1]==c2[-1]:
                        flag=True
                        mark.append('n')
                        n+=1
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer): 
                                t+=1
                            else:    
                                t44.append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                f44.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                    c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                    if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                        flag=True
                        mark.append('\"')
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t44.append(ref1)
                        else: # factual
                            x+=1
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f44.append(ref1)
                if flag==False:
                    uni=ref.strip().split('\\\\\\\\u')
                    if len(uni)>1 and "d Unreliability and Electricity Prices: Th" not in ref:
                        uni=uni[-2][4:]
                        length=len(uni)
                        #if ref1.strip()[-length:] == uni:
                        if uni in ref1.strip():
                            flag = True
                            mark.append('u')
                            mark.append(ref)
                            n+=1
                            if 'true' in label: #hallucination
                                x+=1
                                if identifyConflict(answer): 
                                    t+=1
                                else:    
                                    t44.append(ref1)
                            else: # factual
                                if identifyConflict(answer): 
                                    f+=1
                                else:    
                                    f44.append(ref1)
        if n-pre!=1:
            print(mark)
            print(n-pre)
            print(ref1)
            print()
print(x)
print(len(t1))
print(t)
print(num[-1]-t,(num[-1]-t)/num[-1])
print(f,f/num[-1])
print('matched num=',n)

[]
2
 https://www.thegwpf.com/european-climate-declaration/ The European Climate Declaration (ECD) is a public statement on climate change signed by over 500 scientists and professionals in climate and related fields from Europe and beyond. The declaration asserts that a human-caused climate emergency does not exist and emphasizes the need for a more balanced and nuanced approach to climate policy. It highlights the benefits of carbon dioxide for plant growth and the lack of evidence supporting extreme weather events being caused by human activity. The ECD acknowledges the potential for climate variability but stresses that natural factors such as solar activity and changes in global and regional temperature patterns are also important drivers of climate change. In conclusion, this reference supports a dissenting view on climate change and its causes, challenging the widely accepted position that human activities, particularly burning fossil fuels, are primarily responsible for global 

In [522]:
t=0
f=0
t5=[]
f5=[]
n=0
x=0
with open(filePath[0]+"dataset_0.9.jsonl") as file1:
    for line1 in file1:
        label=line1.split("label: ")[-1]
        ref1=line1.split("Response: ")[0].split("Reference: ")
        if len(ref1)==2:
            ref1=ref1[1]
        else:
            ref1=ref1[1]+"Reference: "+ref1[2]
        pre=n
        flag=False
        ref1=ref1[:-8]
        with open("detections_5_3.jsonl") as file:
            mark=[]
            for line in file:
                answer=line.split("Result: ")[1]
                ref=line.split("P2: ")[0].split("P1: ")[1]
                ref=ref.encode().decode('unicode_escape')
                ref=ref[1:-15]
                if "The Real Inconvenient Truth" in ref:
                    ref+="ng"
                if ref.strip()==ref1.strip():
                    flag=True
                    n+=1
                    if 'true' in label: #hallucination
                        x+=1
                        if identifyConflict(answer):
                            t+=1
                        else:    
                            t5.append(ref1)
                    else: # factual
                        if identifyConflict(answer): 
                            f+=1
                        else:    
                            f5.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split(",")[0]
                    c2=ref1.strip().split(",")[0]
                    if c1.strip()==c2.strip():
                        flag=True
                        n+=1
                        mark.append(',')
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer): 
                                t+=1
                            else:    
                                t5.append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                f5.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\n")
                    c2=ref1.strip().split("\\\\n")
                    if len(c1)>2 and len(c2)>2 and c1[-1]==c2[-1]:
                        flag=True
                        mark.append('n')
                        n+=1
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer): 
                                t+=1
                            else:    
                                t5.append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                f5.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                    c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                    if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                        flag=True
                        mark.append('\"')
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t5.append(ref1)
                        else: # factual
                            x+=1
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f5.append(ref1)
                if flag==False:
                    uni=ref.strip().split('\\\\\\\\u')
                    if len(uni)>1 and "d Unreliability and Electricity Prices: Th" not in ref:
                        uni=uni[-2][4:]
                        length=len(uni)
                        #if ref1.strip()[-length:] == uni:
                        if uni in ref1.strip():
                            flag = True
                            mark.append('u')
                            mark.append(ref)
                            n+=1
                            if 'true' in label: #hallucination
                                x+=1
                                if identifyConflict(answer): 
                                    t+=1
                                else:    
                                    t5.append(ref1)
                            else: # factual
                                if identifyConflict(answer): 
                                    f+=1
                                else:    
                                    f5.append(ref1)
        if n-pre!=1:
            print(mark)
            print(n-pre)
            print(ref1)
            print()
print(x)
print(len(t1))
print(t)
print(num[-1]-t,(num[-1]-t)/num[-1])
print(f,f/num[-1])
print('matched num=',n)

[]
2
 https://www.thegwpf.com/european-climate-declaration/ The European Climate Declaration (ECD) is a public statement on climate change signed by over 500 scientists and professionals in climate and related fields from Europe and beyond. The declaration asserts that a human-caused climate emergency does not exist and emphasizes the need for a more balanced and nuanced approach to climate policy. It highlights the benefits of carbon dioxide for plant growth and the lack of evidence supporting extreme weather events being caused by human activity. The ECD acknowledges the potential for climate variability but stresses that natural factors such as solar activity and changes in global and regional temperature patterns are also important drivers of climate change. In conclusion, this reference supports a dissenting view on climate change and its causes, challenging the widely accepted position that human activities, particularly burning fossil fuels, are primarily responsible for global 

In [523]:
t=0
f=0
t55=[]
f55=[]
n=0
x=0
with open(filePath[0]+"dataset_0.9.jsonl") as file1:
    for line1 in file1:
        label=line1.split("label: ")[-1]
        ref1=line1.split("Response: ")[0].split("Reference: ")
        if len(ref1)==2:
            ref1=ref1[1]
        else:
            ref1=ref1[1]+"Reference: "+ref1[2]
        pre=n
        flag=False
        ref1=ref1[:-8]
        with open(filePath[0]+"detections_5_1.jsonl") as file:
            mark=[]
            for line in file:
                answer=line.split("Result: ")[1]
                ref=line.split("P2: ")[0].split("P1: ")[1]
                ref=ref.encode().decode('unicode_escape')
                ref=ref[1:-15]
                if "The Real Inconvenient Truth" in ref:
                    ref+="ng"
                if ref.strip()==ref1.strip():
                    flag=True
                    n+=1
                    if 'true' in label: #hallucination
                        x+=1
                        if identifyConflict(answer):
                            t+=1
                        else:    
                            t55.append(ref1)
                    else: # factual
                        if identifyConflict(answer): 
                            f+=1
                        else:    
                            f55.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split(",")[0]
                    c2=ref1.strip().split(",")[0]
                    if c1.strip()==c2.strip():
                        flag=True
                        n+=1
                        mark.append(',')
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer): 
                                t+=1
                            else:    
                                t55.append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                f55.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\n")
                    c2=ref1.strip().split("\\\\n")
                    if len(c1)>2 and len(c2)>2 and c1[-1]==c2[-1]:
                        flag=True
                        mark.append('n')
                        n+=1
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer): 
                                t+=1
                            else:    
                                t55.append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                f55.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                    c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                    if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                        flag=True
                        mark.append('\"')
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t55.append(ref1)
                        else: # factual
                            x+=1
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f55.append(ref1)
                if flag==False:
                    uni=ref.strip().split('\\\\\\\\u')
                    if len(uni)>1 and "d Unreliability and Electricity Prices: Th" not in ref:
                        uni=uni[-2][4:]
                        length=len(uni)
                        #if ref1.strip()[-length:] == uni:
                        if uni in ref1.strip():
                            flag = True
                            mark.append('u')
                            mark.append(ref)
                            n+=1
                            if 'true' in label: #hallucination
                                x+=1
                                if identifyConflict(answer): 
                                    t+=1
                                else:    
                                    t55.append(ref1)
                            else: # factual
                                if identifyConflict(answer): 
                                    f+=1
                                else:    
                                    f55.append(ref1)
        if n-pre!=1:
            print(mark)
            print(n-pre)
            print(ref1)
            print()
print(x)
print(len(t1))
print(t)
print(num[-1]-t,(num[-1]-t)/num[-1])
print(f,f/num[-1])
print('matched num=',n)

[]
2
 https://www.thegwpf.com/european-climate-declaration/ The European Climate Declaration (ECD) is a public statement on climate change signed by over 500 scientists and professionals in climate and related fields from Europe and beyond. The declaration asserts that a human-caused climate emergency does not exist and emphasizes the need for a more balanced and nuanced approach to climate policy. It highlights the benefits of carbon dioxide for plant growth and the lack of evidence supporting extreme weather events being caused by human activity. The ECD acknowledges the potential for climate variability but stresses that natural factors such as solar activity and changes in global and regional temperature patterns are also important drivers of climate change. In conclusion, this reference supports a dissenting view on climate change and its causes, challenging the widely accepted position that human activities, particularly burning fossil fuels, are primarily responsible for global 

In [552]:
t=0
f=0
t6=[]
f6=[]
n=0
x=0
with open(filePath[0]+"dataset_0.9.jsonl") as file1:
    for line1 in file1:
        label=line1.split("label: ")[-1]
        ref1=line1.split("Response: ")[0].split("Reference: ")
        if len(ref1)==2:
            ref1=ref1[1]
        else:
            ref1=ref1[1]+"Reference: "+ref1[2]
        pre=n
        flag=False
        ref1=ref1[:-8]
        with open("detections_6_3.jsonl") as file:
            mark=[]
            for line in file:
                answer=line.split("Result: ")[1]
                ref=line.split("P2: ")[0].split("P1: ")[1]
                ref=ref.encode().decode('unicode_escape')
                ref=ref[1:-15]
                if "The Real Inconvenient Truth" in ref:
                    ref+="ng"
                if ref.strip()==ref1.strip():
                    flag=True
                    n+=1
                    if 'true' in label: #hallucination
                        x+=1
                        if identifyConflict(answer):
                            t+=1
                        else:    
                            t6.append(ref1)
                    else: # factual
                        if identifyConflict(answer): 
                            f+=1
                        else:    
                            f6.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split(",")[0]
                    c2=ref1.strip().split(",")[0]
                    if c1.strip()==c2.strip():
                        flag=True
                        n+=1
                        mark.append(',')
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer): 
                                t+=1
                            else:    
                                t6.append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                f6.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\n")
                    c2=ref1.strip().split("\\\\n")
                    if len(c1)>2 and len(c2)>2 and c1[-1]==c2[-1]:
                        flag=True
                        mark.append('n')
                        n+=1
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer): 
                                t+=1
                            else:    
                                t6.append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                f6.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                    c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                    if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                        flag=True
                        mark.append('\"')
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t6.append(ref1)
                        else: # factual
                            x+=1
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f6.append(ref1)
                if flag==False:
                    uni=ref.strip().split('\\\\\\\\u')
                    if len(uni)>1 and "d Unreliability and Electricity Prices: Th" not in ref:
                        uni=uni[-2][4:]
                        length=len(uni)
                        #if ref1.strip()[-length:] == uni:
                        if uni in ref1.strip():
                            flag = True
                            mark.append('u')
                            mark.append(ref)
                            n+=1
                            if 'true' in label: #hallucination
                                x+=1
                                if identifyConflict(answer): 
                                    t+=1
                                else:    
                                    t6.append(ref1)
                            else: # factual
                                if identifyConflict(answer): 
                                    f+=1
                                else:    
                                    f6.append(ref1)
        if n-pre!=1:
            print(mark)
            print(n-pre)
            print(ref1)
            print()
print(x)
print(len(t1))
print(t)
print(num[-1]-t,(num[-1]-t)/num[-1])
print(f,f/num[-1])
print('matched num=',n)

[]
2
 https://www.thegwpf.com/european-climate-declaration/ The European Climate Declaration (ECD) is a public statement on climate change signed by over 500 scientists and professionals in climate and related fields from Europe and beyond. The declaration asserts that a human-caused climate emergency does not exist and emphasizes the need for a more balanced and nuanced approach to climate policy. It highlights the benefits of carbon dioxide for plant growth and the lack of evidence supporting extreme weather events being caused by human activity. The ECD acknowledges the potential for climate variability but stresses that natural factors such as solar activity and changes in global and regional temperature patterns are also important drivers of climate change. In conclusion, this reference supports a dissenting view on climate change and its causes, challenging the widely accepted position that human activities, particularly burning fossil fuels, are primarily responsible for global 

In [512]:
with open('detections_6_3.jsonl','w') as twofile:
    with open("llama2_data/climate+0.9/dataset_0.9.jsonl") as file1:
        for line1 in file1:
            label=line1.split("label: ")[-1]
            ref1=line1.split("Response: ")[0].split("Reference: ")
            if len(ref1)==2:
                ref1=ref1[1]
            else:
                ref1=ref1[1]+"Reference: "+ref1[2]
            pre=n
            flag=False
            ref1=ref1[:-8]
            if (ref1 in f6 and "false" in label) or (ref1 not in t6 and "true" in label):
                with open("llama2_data/climate+0.9/detections_6_2.jsonl") as file:
                    flag=False
                    for line in file:
                        answer=line.split("Result: ")[1]
                        ref=line.split("P2: ")[0].split("P1: ")[1]
                        ref=ref.encode().decode('unicode_escape')
                        ref=ref[1:-15]
                        if "The Real Inconvenient Truth" in ref:
                            ref+="ng"
                            if ref.strip()==ref1.strip():
                                flag=True
                                twofile.write(line)
                        if flag==False:
                            # 
                            c1=ref.strip().split(",")[0]
                            c2=ref1.strip().split(",")[0]
                            if c1.strip()==c2.strip():
                                flag=True
                                twofile.write(line)
                        if flag==False:
                            # 
                            c1=ref.strip().split("\\\\n")
                            c2=ref1.strip().split("\\\\n")
                            if len(c1)>2 and len(c2)>2 and c1[-1]==c2[-1]:
                                flag=True
                                twofile.write(line)
                        if flag==False:
                            # 
                            c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                            c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                            if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                                flag=True
                                twofile.write(line)
                        if flag==False:
                            uni=ref.strip().split('\\\\\\\\u')
                            if len(uni)>1 and "d Unreliability and Electricity Prices: Th" not in ref:
                                uni=uni[-2][4:]
                                length=len(uni)
                                #if ref1.strip()[-length:] == uni:
                                if uni in ref1.strip():
                                    flag = True
                                    twofile.write(line)
                    if flag==False:
                        print('one error')
            else:
                print('xxx')
                with open('llama2_data/climate+0.9/detections_6.jsonl') as file2:
                    flag=False
                    for line2 in file2:
                        answer=line2.split("Result: ")[1]
                        ref=line2.split("P2: ")[0].split("P1: ")[1]
                        ref=ref.encode().decode('unicode_escape')
                        ref=ref[1:-15]
                        if "The Real Inconvenient Truth" in ref:
                            ref+="ng"
                            if ref.strip()==ref1.strip():
                                flag=True
                                twofile.write(line2)
                        if flag==False:
                            # 
                            c1=ref.strip().split(",")[0]
                            c2=ref1.strip().split(",")[0]
                            if c1.strip()==c2.strip():
                                flag=True
                                twofile.write(line2)
                        if flag==False:
                            # 
                            c1=ref.strip().split("\\\\n")
                            c2=ref1.strip().split("\\\\n")
                            if len(c1)>2 and len(c2)>2 and c1[-1]==c2[-1]:
                                flag=True
                                twofile.write(line2)
                        if flag==False:
                            # 
                            c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                            c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                            if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                                flag=True
                                twofile.write(line2)
                        if flag==False:
                            uni=ref.strip().split('\\\\\\\\u')
                            if len(uni)>1 and "d Unreliability and Electricity Prices: Th" not in ref:
                                uni=uni[-2][4:]
                                length=len(uni)
                                if uni in ref1.strip():
                                    flag = True
                                    twofile.write(line2)
                    if flag==False:
                        print('one error')

xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
one error
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx


In [525]:
t=0
f=0
t66=[]
f66=[]
n=0
x=0
with open(filePath[0]+"dataset_0.9.jsonl") as file1:
    for line1 in file1:
        label=line1.split("label: ")[-1]
        ref1=line1.split("Response: ")[0].split("Reference: ")
        if len(ref1)==2:
            ref1=ref1[1]
        else:
            ref1=ref1[1]+"Reference: "+ref1[2]
        pre=n
        flag=False
        ref1=ref1[:-8]
        with open(filePath[0]+"detections_6_1.jsonl") as file:
            mark=[]
            for line in file:
                answer=line.split("Result: ")[1]
                ref=line.split("P2: ")[0].split("P1: ")[1]
                ref=ref.encode().decode('unicode_escape')
                ref=ref[1:-15]
                if "The Real Inconvenient Truth" in ref:
                    ref+="ng"
                if ref.strip()==ref1.strip():
                    flag=True
                    n+=1
                    if 'true' in label: #hallucination
                        x+=1
                        if identifyConflict(answer):
                            t+=1
                        else:    
                            t66.append(ref1)
                    else: # factual
                        if identifyConflict(answer): 
                            f+=1
                        else:    
                            f66.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split(",")[0]
                    c2=ref1.strip().split(",")[0]
                    if c1.strip()==c2.strip():
                        flag=True
                        n+=1
                        mark.append(',')
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer): 
                                t+=1
                            else:    
                                t66.append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                f66.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\n")
                    c2=ref1.strip().split("\\\\n")
                    if len(c1)>2 and len(c2)>2 and c1[-1]==c2[-1]:
                        flag=True
                        mark.append('n')
                        n+=1
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer): 
                                t+=1
                            else:    
                                t66.append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                f66.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                    c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                    if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                        flag=True
                        mark.append('\"')
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t66.append(ref1)
                        else: # factual
                            x+=1
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f66.append(ref1)
                if flag==False:
                    uni=ref.strip().split('\\\\\\\\u')
                    if len(uni)>1 and "d Unreliability and Electricity Prices: Th" not in ref:
                        uni=uni[-2][4:]
                        length=len(uni)
                        #if ref1.strip()[-length:] == uni:
                        if uni in ref1.strip():
                            flag = True
                            mark.append('u')
                            mark.append(ref)
                            n+=1
                            if 'true' in label: #hallucination
                                x+=1
                                if identifyConflict(answer): 
                                    t+=1
                                else:    
                                    t66.append(ref1)
                            else: # factual
                                if identifyConflict(answer): 
                                    f+=1
                                else:    
                                    f66.append(ref1)
        if n-pre!=1:
            print(mark)
            print(n-pre)
            print(ref1)
            print()
print(x)
print(len(t1))
print(t)
print(num[-1]-t,(num[-1]-t)/num[-1])
print(f,f/num[-1])
print('matched num=',n)

[]
2
 https://www.thegwpf.com/european-climate-declaration/ The European Climate Declaration (ECD) is a public statement on climate change signed by over 500 scientists and professionals in climate and related fields from Europe and beyond. The declaration asserts that a human-caused climate emergency does not exist and emphasizes the need for a more balanced and nuanced approach to climate policy. It highlights the benefits of carbon dioxide for plant growth and the lack of evidence supporting extreme weather events being caused by human activity. The ECD acknowledges the potential for climate variability but stresses that natural factors such as solar activity and changes in global and regional temperature patterns are also important drivers of climate change. In conclusion, this reference supports a dissenting view on climate change and its causes, challenging the widely accepted position that human activities, particularly burning fossil fuels, are primarily responsible for global 

In [526]:
t=0
f=0
t7=[]
f7=[]
n=0
x=0
with open(filePath[0]+"dataset_0.9.jsonl") as file1:
    for line1 in file1:
        label=line1.split("label: ")[-1]
        ref1=line1.split("Response: ")[0].split("Reference: ")
        if len(ref1)==2:
            ref1=ref1[1]
        else:
            ref1=ref1[1]+"Reference: "+ref1[2]
        pre=n
        flag=False
        ref1=ref1[:-8]
        with open(filePath[0]+"detections_7.jsonl") as file:
            mark=[]
            for line in file:
                answer=line.split("Result: ")[1]
                ref=line.split("P2: ")[0].split("P1: ")[1]
                ref=ref.encode().decode('unicode_escape')
                ref=ref[1:-15]
                if "The Real Inconvenient Truth" in ref:
                    ref+="ng"
                if ref.strip()==ref1.strip():
                    flag=True
                    n+=1
                    if 'true' in label: #hallucination
                        x+=1
                        if identifyConflict(answer):
                            t+=1
                        else:    
                            t7.append(ref1)
                    else: # factual
                        if identifyConflict(answer): 
                            f+=1
                        else:    
                            f7.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split(",")[0]
                    c2=ref1.strip().split(",")[0]
                    if c1.strip()==c2.strip():
                        flag=True
                        n+=1
                        mark.append(',')
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer): 
                                t+=1
                            else:    
                                t7.append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                f7.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\n")
                    c2=ref1.strip().split("\\\\n")
                    if len(c1)>2 and len(c2)>2 and c1[-1]==c2[-1]:
                        flag=True
                        mark.append('n')
                        n+=1
                        if 'true' in label: #hallucination
                            x+=1
                            if identifyConflict(answer): 
                                t+=1
                            else:    
                                t7.append(ref1)
                        else: # factual
                            if identifyConflict(answer): 
                                f+=1
                            else:    
                                f7.append(ref1)
                if flag==False:
                    # 
                    c1=ref.strip().split("\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"")
                    c2=ref1.strip().split("\\\\\\\\\\\\\\\"")
                    if len(c1)>2 and len(c2)>2 and c1[1]==c2[1]:
                        flag=True
                        mark.append('\"')
                        n+=1
                        if 'true' in label: #hallucination
                            if identifyConflict(answer):
                                t+=1
                            else:    
                                t7.append(ref1)
                        else: # factual
                            x+=1
                            if identifyConflict(answer):
                                f+=1
                            else:    
                                f7.append(ref1)
                if flag==False:
                    uni=ref.strip().split('\\\\\\\\u')
                    if len(uni)>1 and "d Unreliability and Electricity Prices: Th" not in ref:
                        uni=uni[-2][4:]
                        length=len(uni)
                        #if ref1.strip()[-length:] == uni:
                        if uni in ref1.strip():
                            flag = True
                            mark.append('u')
                            mark.append(ref)
                            n+=1
                            if 'true' in label: #hallucination
                                x+=1
                                if identifyConflict(answer): 
                                    t+=1
                                else:    
                                    t7.append(ref1)
                            else: # factual
                                if identifyConflict(answer): 
                                    f+=1
                                else:    
                                    f7.append(ref1)
        if n-pre!=1:
            print(mark)
            print(n-pre)
            print(ref1)
            print()
print(x)
print(len(t1))
print(t)
print(num[-1]-t,(num[-1]-t)/num[-1])
print(f,f/num[-1])
print('matched num=',n)

[]
2
 https://www.thegwpf.com/european-climate-declaration/ The European Climate Declaration (ECD) is a public statement on climate change signed by over 500 scientists and professionals in climate and related fields from Europe and beyond. The declaration asserts that a human-caused climate emergency does not exist and emphasizes the need for a more balanced and nuanced approach to climate policy. It highlights the benefits of carbon dioxide for plant growth and the lack of evidence supporting extreme weather events being caused by human activity. The ECD acknowledges the potential for climate variability but stresses that natural factors such as solar activity and changes in global and regional temperature patterns are also important drivers of climate change. In conclusion, this reference supports a dissenting view on climate change and its causes, challenging the widely accepted position that human activities, particularly burning fossil fuels, are primarily responsible for global 

In [532]:
print(len(t7))

168


In [531]:
len(set(t1+t7))

169

In [557]:
tlist=[t1,t2,t3,t4,t5,t6,t7,t11,t22,t33,t44,t55,t66]
flist=[f1,f2,f3,f4,f5,f6,f7,f11,f22,f33,f44,f55,f66]
#      0   1  2  3  4  5  6  7   8   9   10  11  12
minV=0
for kk in range(13-9):
    for jj in range(kk+1,13-8):
        for ii in range(jj+1,13-7):
            for i in range(ii+1,13-6):
                for j in range(i+1,13-5):
                     for k in range(j+1,13-4):
                        for m in range(k+1,13-3):
                            for n in range(m+1,13-2):
                                for q in range(n+1,13-1):
                                    for w in range(q+1,13-0):
                                        #for z in range(w+1,13-0):
                                            #for y in range(z+1,13-0):
                                                a=tlist[kk]+tlist[jj]+tlist[ii]+tlist[i]+tlist[j]+tlist[k]+tlist[m]+tlist[n]+tlist[q]+tlist[w]#+tlist[z]+tlist[y]+tlist[p] 
                                                a=list(set(a))
                                                aa=len(a)
                                                a=flist[kk]+flist[jj]+flist[ii]+flist[i]+flist[j]+flist[k]+flist[m]+flist[n]+flist[q]+flist[w]#+flist[z]+flist[y]+flist[p] 
                                                a=list(set(a))
                                                bb=len(a)
                                                TN=(num[-1]-bb)/(num[-1]*2)
                                                TP=aa/(num[-1]*2)
                                                FN=(num[-1]-aa)/(num[-1]*2)
                                                FP=bb/(num[-1]*2)
                                                F1=2*TP/(2*TP+FP+FN)
                                                if TN+TP>=minV:
                                                    print(kk,jj,ii,i)#,j,k,m,n,q,w,z,y,p)
                                                    print(TN+TP)
                                                    print(F1)
                                                    minV=TN+TP

0 1 2 3
0.4945652173913044
0.654275092936803
0 1 2 3
0.4945652173913044
0.654275092936803
0 1 2 3
0.4945652173913044
0.654275092936803
0 1 2 3
0.4945652173913044
0.654275092936803
0 1 2 3
0.4945652173913044
0.654275092936803
0 1 2 3
0.4945652173913044
0.654275092936803
0 1 2 3
0.4945652173913044
0.654275092936803
0 1 2 3
0.4972826086956522
0.6554934823091247
0 1 2 3
0.4972826086956522
0.6554934823091247
0 1 2 3
0.4972826086956522
0.6554934823091247
0 1 2 3
0.4972826086956522
0.6554934823091247
0 1 2 3
0.5
0.6567164179104477
0 1 2 3
0.5
0.6567164179104477
0 1 2 3
0.5
0.6567164179104477
0 1 2 3
0.5
0.6567164179104477
0 1 2 3
0.5
0.6567164179104477
0 1 2 3
0.5
0.6554307116104869
0 2 3 4
0.5
0.6567164179104477
0 2 3 4
0.5
0.6567164179104477
0 2 3 4
0.5
0.6567164179104477
0 2 3 4
0.5
0.6567164179104477
